In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import TensorDataset, DataLoader, random_split
import numpy as np
import matplotlib.pyplot as plt

import sys
sys.path.append("../src/")
import importlib
import structures
importlib.reload(structures)
from structures import *
import losses
importlib.reload(losses)
from losses import *
import trainer
importlib.reload(trainer)
from trainer import *
#dtype = torch.float32
dtype = torch.double
torch.set_default_dtype(dtype)
torch.autograd.set_detect_anomaly(True)
# Set the device to CUDA if available, otherwise CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

import random
sys.path.append("/mnt/home/yjo10/ceph/myutils")
import plt_utils as pu

Using device: cuda
[2025-04-13 20:12:06,601] [INFO] [real_accelerator.py:222:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/mnt/sw/nix/store/r3bwp9b2501bv77y6g1nwkb483p0y9z2-cuda-12.3.2/lib64/libcufile.so: undefined reference to `dlvsym'
/mnt/sw/nix/store/r3bwp9b2501bv77y6g1nwkb483p0y9z2-cuda-12.3.2/lib64/libcufile.so: undefined reference to `dlopen'
/mnt/sw/nix/store/r3bwp9b2501bv77y6g1nwkb483p0y9z2-cuda-12.3.2/lib64/libcufile.so: undefined reference to `dlclose'
/mnt/sw/nix/store/r3bwp9b2501bv77y6g1nwkb483p0y9z2-cuda-12.3.2/lib64/libcufile.so: undefined reference to `dlerror'
/mnt/sw/nix/store/r3bwp9b2501bv77y6g1nwkb483p0y9z2-cuda-12.3.2/lib64/libcufile.so: undefined reference to `dlsym'
/mnt/sw/nix/store/r3bwp9b2501bv77y6g1nwkb483p0y9z2-cuda-12.3.2/lib64/libcufile.so: undefined reference to `shm_open'
/mnt/sw/nix/store/r3bwp9b2501bv77y6g1nwkb483p0y9z2-cuda-12.3.2/lib64/libcufile.so: undefined reference to `shm_unlink'
collect2: error: ld returned 1 exit status


# Two Body

## OLD Preliminary version with positions, velocity, and acceleration

In [13]:
#from src.trainer import num_samples

# Example unsupervised data: 100 samples with 10 features each
data = np.load("../data/two_body_train_data.npy")
num_samples = data.shape[0]
num_samples_start = int(data.shape[0]*0.9)
#num_samples_start = 0
num_samples_end   = data.shape[0]
num_samples = num_samples_end - num_samples_start
num_features = data.shape[1]

# Create a tensor of features only
data_org = torch.tensor(data[num_samples_start:num_samples_end,:], dtype=torch.double).to(device)
data = data_org.clone()


# Normalize the data
data_min = data_org[:,:26].min(axis=0, keepdim=True).values
data_max = data_org[:,:26].max(axis=0, keepdim=True).values
data[:,:26] = (data_org[:,:26] - data_min) / (data_max - data_min)

# Wrap the feature tensor in a TensorDataset
# Each item from the dataset will be a tuple containing one tensor (X[i],)
dataset = TensorDataset(data)

# Define the proportion for the test set (e.g., 20%)
test_ratio = 0.2
num_test = int(num_samples * test_ratio)
num_train = num_samples - num_test

# Use random_split to split the dataset into train and test subsets
train_dataset, test_dataset = random_split(dataset, [num_train, num_test])

# Create DataLoaders for the training and testing datasets
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

### Loss function test


In [14]:
import structures
importlib.reload(structures)
from structures import *
import losses
importlib.reload(losses)
from losses import *
criterion = CustomizableLoss(nParticle=2, nAttribute=13, nBatch=batch_size,alpha=0.1, beta=0.1, gamma=10.0, TargetEnergyError=TargetEnergyError,
                            data_min=data_min, data_max=data_max,device=device)
X = data[:2,:]
print(data_org[:2,:])#, data[:2,:])
loss, loss_terms = criterion(output[:2,:], X)

NameError: name 'CustomizableLoss' is not defined

### Training

In [ ]:
import structures
importlib.reload(structures)
from structures import *
import losses
importlib.reload(losses)
from losses import *
import trainer
importlib.reload(trainer)
from trainer import *


input_size = 13     # Number of input features
hidden_size = 500     # Number of hidden neurons
output_size = 2     # Number of output 
TargetEnergyError = 1e-5

# Instantiate the model
try: 
    del model
    print("model deleted")
except:
    pass

#model = SimpleNN(input_size, hidden_size, output_size).to(device)
model = FullyConnectedNN(input_dim=input_size, output_dim=output_size, hidden_dims=[64, 64], activation='relu', dropout=0.0, output_positive=True).to(device)


# Define the loss function (CrossEntropyLoss is common for classification tasks)
criterion = CustomizableLoss(nParticle=2, nAttribute=13, nBatch=batch_size,alpha=0.1, beta=0.1, gamma=10.0, TargetEnergyError=TargetEnergyError,
                            data_min=data_min, data_max=data_max,device=device)

# Define the optimizer (Stochastic Gradient Descent in this example)
#optimizer = optim.SGD(model.parameters(), lr=0.001)
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Paths to save best and last models
base_path = "/mnt/home/yjo10/projects/AITimeStepper/data/models/two_body/"
best_val_path = base_path + "best_model_loss.pth"
best_timestep_path = base_path + "best_model_timestep.pth"
#last_model_path = base_path + f"last_model_{epoch}.pth"
best_val_loss = float("inf")
best_largest_timestep = 0

example_input = torch.randn(1, input_size).to(device)

# Training routine
num_epochs = 1000
for epoch in range(num_epochs):

    train_loss = \
        train_one_epoch(model, optimizer, criterion, train_loader, input_size, device)
    test_loss, energy_error, energy_error_fiducial, time_step, time_step_fiducial = \
        validate(model, criterion, test_loader, input_size, device)

    if epoch > 5:
        # Save the best model based on validation loss
        if test_loss < best_val_loss:
            best_val_loss = test_loss
            torch.save(model.state_dict(), best_val_path)
            traced_model = torch.jit.trace(model, example_input)
            traced_model.save(base_path + f"c++/best_model_loss.pt")
            print(f"--> Best model saved at epoch {epoch} with val loss {best_val_loss:.4e}")
        
        # Save the best model based on validation largest timestep
        if time_step > best_largest_timestep:
            best_largest_timestep = time_step
            torch.save(model.state_dict(), best_timestep_path)
            traced_model = torch.jit.trace(model, example_input)
            traced_model.save(base_path + f"c++/best_model_timestep.pt")
            print(f"--> Best model saved at epoch {epoch} with largest timestep {best_largest_timestep:.4e}")

    # Save the last model after every epoch (or just once at the end)
    torch.save(model.state_dict(), base_path + f"model_{epoch}.pth")
    traced_model = torch.jit.trace(model, example_input)
    traced_model.save(base_path + f"c++/model_{epoch}.pt")

    print(f"Epoch [{epoch}/{num_epochs}], Train Loss: {train_loss:.4e}, Test Loss: {test_loss:.4e}, Energy Loss: {energy_error:.4e}/{energy_error_fiducial:.4e}, Time step: {time_step:.4e}/{time_step_fiducial:.4e}")

## NEW version with magnitudes of velocity and acceleration, and mass

In [ ]:
def normalizer(_data):
    _max = torch.abs(_data.max())
    data_max = torch.ones(_data.size(), device=device)*_max
    _data = _data/data_max
    return _data, _max
    
def denormalizer(_data, _max):
    data_max = torch.ones(_data.size(), device=device)*_max
    return _data*data_max

# Example unsupervised data: 100 samples with 10 features each
data = np.load("../data/two_body_train_data_new.npy")
num_samples = data.shape[0]
num_samples_start = int(data.shape[0]*0.8)
num_samples_start = 0
num_samples_end   = data.shape[0]
num_samples = num_samples_end - num_samples_start
num_features = data.shape[1]

# Placeholder for input (magnitudes of velocities and accelerations)
data_org = torch.tensor(data[num_samples_start:num_samples_end,:], dtype=dtype).to(device)
data = torch.empty((num_samples,6+num_features), dtype=dtype).to(device)
data[:,6:] = data_org


# Magnitudes vel, acc, mass
temp, tmax = normalizer(data_org[:,4:7])
vel = denormalizer(torch.norm(temp, p=2, dim=1),tmax)
acc = torch.empty((num_samples,4), dtype=dtype).to(device)

temp,tmax = normalizer(data_org[:,7:10])
acc[:,0] = denormalizer(torch.norm(temp, p=2, dim=1),tmax)
temp,tmax = normalizer(data_org[:,10:13])
acc[:,1] = denormalizer(torch.norm(temp, p=2, dim=1),tmax)
temp,tmax = normalizer(data_org[:,13:16])
acc[:,2] = denormalizer(torch.norm(temp, p=2, dim=1),tmax)
temp,tmax = normalizer(data_org[:,16:19])
acc[:,3] = denormalizer(torch.norm(temp, p=2, dim=1),tmax)
mass = data_org[:,0]

data[:,0] = vel
data[:,1:5] = acc
data[:,5] = mass

# Normalize the data
eps = 1e-5
data_min = data[:,:6].min(axis=0, keepdim=True).values
data_max = data[:,:6].max(axis=0, keepdim=True).values
data[:,:6] = (data[:,:6] - data_min) / (data_max - data_min)+eps


# Wrap the feature tensor in a TensorDataset
# Each item from the dataset will be a tuple containing one tensor (X[i],)
dataset = TensorDataset(data)

# Define the proportion for the test set (e.g., 20%)
test_ratio = 0.2
num_test = int(num_samples * test_ratio)
num_train = num_samples - num_test

# Use random_split to split the dataset into train and test subsets
# Define generator with a fixed seed
generator = torch.Generator().manual_seed(42)
train_dataset, test_dataset = random_split(dataset, [num_train, num_test], generator=generator)

# Create DataLoaders for the training and testing datasets
batch_size = 8
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
print(data.shape)

torch.Size([590, 48])


In [15]:
data[:,:6]

tensor([[2.2343e-01, 4.0010e-02, 3.2632e-03, 3.9425e-04, 4.0169e-05, 1.0000e+00],
        [2.2343e-01, 4.0010e-02, 3.2632e-03, 3.9425e-04, 4.0169e-05, 1.0000e+00],
        [2.2344e-01, 4.0010e-02, 3.2632e-03, 6.5004e-05, 1.1570e-05, 1.0000e+00],
        ...,
        [3.1449e-01, 1.0010e-02, 2.9902e-04, 1.8662e-05, 1.0249e-05, 1.0000e-05],
        [1.8249e-03, 2.1083e-05, 1.0070e-05, 1.0000e-05, 1.0000e-05, 1.0000e+00],
        [2.0696e-01, 1.1211e-03, 1.7046e-05, 1.0042e-05, 1.0000e-05, 1.0000e-05]],
       device='cuda:0')

In [ ]:
data.max

### Loss function test


In [6]:
import structures
importlib.reload(structures)
from structures import *
import losses
importlib.reload(losses)
from losses import *
TargetEnergyError = 1e-8
criterion = CustomizableLoss3DM(nParticle=2, nAttribute=13, nBatch=batch_size,alpha=0.1, beta=0.1, gamma=10.0, TargetEnergyError=TargetEnergyError,
                            data_min=data_min, data_max=data_max,device=device)
X = data[:2,:]
print(data_org[:2,:])#, data[:2,:])
#loss, loss_terms = criterion(output[:2,:], X)

tensor([[ 3.5110e+04,  1.2500e-02,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         -8.3790e+02,  0.0000e+00, -5.6160e+07,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  3.7640e+12,  0.0000e+00,  1.7660e+18,  0.0000e+00,
          0.0000e+00,  0.0000e+00, -3.2130e+23,  0.0000e+00,  1.4650e-06,
          3.5110e+04, -1.2500e-02,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          8.3790e+02,  0.0000e+00,  5.6160e+07,  0.0000e+00,  0.0000e+00,
          0.0000e+00, -3.7640e+12,  0.0000e+00, -1.7660e+18,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  3.2130e+23,  0.0000e+00,  1.4650e-06,
          1.0747e-04, -2.4659e+10],
        [ 3.5110e+04, -1.2500e-02,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          8.3790e+02,  0.0000e+00,  5.6160e+07,  0.0000e+00,  0.0000e+00,
          0.0000e+00, -3.7640e+12,  0.0000e+00, -1.7660e+18,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  3.2130e+23,  0.0000e+00,  1.4650e-06,
          3.5110e+04,  1.2500e-02,  0.0000e+00,  0.0000e+00,  0.0000e+00,
  

### Training

In [ ]:
import structures
importlib.reload(structures)
from structures import *
import losses
importlib.reload(losses)
from losses import *
import trainer
importlib.reload(trainer)
from trainer import *

input_mask = np.r_[0:3,5]
print
input_size = len(input_mask)
#hidden_dims = [32,64,64,32]     # Number of hidden neurons
#hidden_dims = [8,16,16,4]     # Number of hidden neurons
hidden_dims = [12,12,6]     # Number of hidden neurons
#hidden_dims = [16,128,32,8]     # Number of hidden neurons
#hidden_dims = [8,8]     # Number of hidden neurons
output_size = 2     # Number of output 
TargetEnergyError = 1e-3
weights = {"time_step":10, "energy_loss":1}

# Instantiate the model
try: 
    del mode
    l
    print("model deleted")
except:
    pass


ds_config = {
    "train_batch_size": batch_size,
    "gradient_accumulation_steps": 1,
    "optimizer": {
        "type": "Adam",
        "params": {"lr": 1e-3, "betas": [0.9, 0.999], "eps": 1e-8}
    },
    "fp32": {
        "enabled": True  # Enables mixed precision training
    },
    "zero_optimization": {
        "stage": 1  # Enable ZeRO Stage 1 for memory optimization
    }
}


#model = SimpleNN(input_size, hidden_size, output_size).to(device)
model = FullyConnectedNN(input_dim=input_size, output_dim=output_size, hidden_dims=hidden_dims, activation='relu', dropout=0.0, output_positive=False).to(device)


model_engine, optimizer, _, _ = deepspeed.initialize(
    model=model,
    model_parameters=model.parameters(),
    config=ds_config
)

# Define the loss function (CrossEntropyLoss is common for classification tasks)
criterion = CustomizableLoss3DM(nParticle=2, nAttribute=20, nBatch=batch_size,alpha=weights['time_step'], beta=weights['energy_loss'], gamma=weights['energy_loss'], TargetEnergyError=TargetEnergyError,
                            data_min=data_min, data_max=data_max,device=device)

# Define the optimizer (Stochastic Gradient Descent in this example)
#optimizer = optim.SGD(model.parameters(), lr=0.001)
#optimizer = optim.Adam(model.parameters(), lr=1e-5)
#optimizer = optim.AdamW(model.parameters(), lr=0.00001)
#optimizer = optim.AdamW(model.parameters(), lr=0.00001)



# Paths to save best and last models
base_path = "/mnt/home/yjo10/projects/AITimeStepper/data/models/two_body/"
best_val_path = base_path + "best_model_loss.pth"
best_timestep_path = base_path + "best_model_timestep.pth"
#last_model_path = base_path + f"last_model_{epoch}.pth"
best_val_loss = float("inf")
best_largest_timestep = 0
with open(base_path+"c++/normalization_factors.txt", "w") as f:
    f.write(f"{data_min[0][0]} {data_min[0][1]} {data_min[0][2]} {data_min[0][3]} {data_min[0][4]} {data_min[0][5]}\n")    
    f.write(f"{data_max[0][0]} {data_max[0][1]} {data_max[0][2]} {data_max[0][3]} {data_max[0][4]} {data_max[0][5]}\n")    

example_input = torch.randn(1, input_size).to(device)
exp_time_step = 1
exp_energy_loss = 1
# Training routine
num_epochs = 1000
lists = {"training_loss":[], "val_loss":[], "training_energy":[], "val_energy":[], "training_time_step":[], "val_time_step":[]}

for epoch in range(num_epochs):

    #train_loss, train_energy = \
    #    train_one_epoch(model, optimizer, criterion, train_loader, input_mask, weights, device)
    train_res = train_one_epoch_deepspeed(model_engine, optimizer, criterion, train_loader, input_mask, weights, device)
        
    #test_loss, energy_error, energy_error_std, energy_error_fiducial, energy_error_fiducial_std, energy_pred, energy_init, time_step, time_step_fiducial = \
    #    validate(model, criterion, test_loader, input_mask, weights, device)
    val_res = validate_deepspeed(model_engine, criterion, test_loader, input_mask, weights, device)

    lists["training_loss"].append(train_res["train_loss"])
    lists["training_energy"].append(train_res["energy_error"])
    lists["training_time_step"].append(train_res["time_step"])

    lists["val_loss"].append(val_res["val_loss"])
    lists["val_energy"].append(val_res["energy_error"])
    lists["val_time_step"].append(val_res["time_step"])

    if epoch > 5:
        # Save the best model based on validation loss
        if val_res['val_loss'] < best_val_loss:
            best_val_loss = val_res['val_loss']
            torch.save(model.state_dict(), best_val_path)
            #traced_model = torch.jit.trace(model, example_input)
            traced_model = torch.jit.script(model)
            traced_model.save(base_path + f"c++/best_model_loss.pt")
            print(f"--> Best model saved at epoch {epoch} with val loss {best_val_loss:.4e}")
        
        # Save the best model based on validation largest timestep
        if val_res['time_step'] > best_largest_timestep:
            best_largest_timestep = val_res['time_step']
            torch.save(model.state_dict(), best_timestep_path)
            #traced_model = torch.jit.trace(model, example_input)
            traced_model = torch.jit.script(model)
            traced_model.save(base_path + f"c++/best_model_timestep.pt")
            print(f"--> Best model saved at epoch {epoch} with largest timestep {best_largest_timestep:.4e}")

    # Save the last model after every epoch (or just once at the end)
    torch.save(model.state_dict(), base_path + f"model_{epoch}.pth")
    #traced_model = torch.jit.trace(model, example_input)
    traced_model = torch.jit.script(model)
    traced_model.save(base_path + f"c++/model_{epoch}.pt")



    fig, axes = pu.generateAxesForMultiplePlots(shape=(1,3),figsize=(10,10),hspace=0.1,wspace=0.1,
                                    gridspec=None)
    for i, (key, value) in enumerate(lists.items()):
        y = i//2
        axes[y].plot(range(epoch+1), value, label=key)
        axes[y].scatter(range(epoch+1), value)
    axes[0].set_yscale("log")
    axes[2].set_yscale("log")

    axes[0].grid()
    axes[1].grid()
    axes[2].grid()


    axes[2].axhline(y=val_res['time_step_fiducial'], c='r',alpha=0.5,zorder=-1,label="Timestep Fiducial")
    axes[1].axhline(y=np.log10(TargetEnergyError), c='r',alpha=0.5,zorder=-1,label="Target energy")
    #plt.legend(bbox_to_anchor=(1.01, 1), loc='upper left', borderaxespad=0.)
    axes[0].legend()
    axes[1].legend()
    axes[2].legend()
    #plt.tight_layout()
    axes[0].set_ylabel("Loss")
    axes[1].set_ylabel("log(Relative Error of Energy)")
    axes[2].set_ylabel("Timestep")
    axes[2].set_xlabel("epoch")
    plt.savefig("learning_curve.png",dpi=100)
    plt.close()



    fig, axes = pu.generateAxesForMultiplePlots(shape=(1,2),figsize=(10,15),hspace=0.1,wspace=0.1,
                                    gridspec=None)
    # Get one batch from the validation loader
    dataiter = iter(train_loader)
    inputs, = next(dataiter)

    # Determine how many samples to take (if batch size < 50, take the whole batch)
    num_samples = min(50, batch_size)

    # Randomly select indices from the batch
    indices = random.sample(range(batch_size), num_samples)
    sampled_inputs = inputs[:,input_mask][indices]
    sampled_targets = inputs[indices,25]

    # Compute model predictions with no gradient computation
    with torch.no_grad():
        sampled_predictions = model(sampled_inputs)

    # Convert tensors to numpy arrays for plotting
    sampled_predictions = sampled_predictions[:,0].cpu().numpy().flatten()
    sampled_targets = sampled_targets.cpu().numpy().flatten()

    # Create sample indices for the x-axis (1st sample, 2nd sample, etc.)
    sample_indices = list(range(1, num_samples + 1))

    # Plot predictions and ground truth vs sample number
    # plt.figure(figsize=(8, 6))
    # Add vertical lines for each data point
    for x in sample_indices:
        axes[0].axvline(x=x, color='grey', linestyle='-', alpha=0.2,zorder=0)
    axes[0].scatter(sample_indices, sampled_targets, marker='o', label='Ground Truth', fc="none", ec="k",zorder=10)
    axes[0].scatter(sample_indices, np.power(10,sampled_predictions), marker='s', label='Predictions', fc="none",ec="r" ,zorder=10)
    axes[0].set_yscale("log")
    axes[0].set_xlabel('Sample Number')
    axes[0].set_ylabel('Value')

    # Get one batch from the validation loader
    dataiter = iter(test_loader)
    inputs, = next(dataiter)

    # Determine how many samples to take (if batch size < 50, take the whole batch)
    num_samples = min(50, batch_size)

    # Randomly select indices from the batch
    indices = random.sample(range(batch_size), num_samples)
    sampled_inputs = inputs[:,input_mask][indices]
    sampled_targets = inputs[indices,25]

    # Compute model predictions with no gradient computation
    with torch.no_grad():
        sampled_predictions = model(sampled_inputs)

    # Convert tensors to numpy arrays for plotting
    sampled_predictions = sampled_predictions[:,0].cpu().numpy().flatten()
    sampled_targets = sampled_targets.cpu().numpy().flatten()

    # Create sample indices for the x-axis (1st sample, 2nd sample, etc.)
    sample_indices = list(range(1, num_samples + 1))

    # Plot predictions and ground truth vs sample number
    # plt.figure(figsize=(8, 6))
    # Add vertical lines for each data point
    for x in sample_indices:
        axes[1].axvline(x=x, color='grey', linestyle='-', alpha=0.2,zorder=0)
    axes[1].scatter(sample_indices, sampled_targets, marker='o', label='Ground Truth', fc="none", ec="k",zorder=10)
    axes[1].scatter(sample_indices, np.power(10,sampled_predictions), marker='s', label='Predictions', fc="none",ec="r" ,zorder=10)
    axes[1].set_yscale("log")
    axes[1].set_xlabel('Sample Number')
    axes[1].set_ylabel('Value')

    axes[0].set_title(f'Epoch [{epoch}/{num_epochs}] Predictions and Ground Truth vs. Sample Number')
    #plt.legend()
    axes[0].grid()
    axes[1].grid()
    plt.legend(bbox_to_anchor=(1.01, 1), loc='upper left', borderaxespad=0.)
    plt.savefig("samples.png")
    plt.close()


    """
    if time_step < time_step_fiducial:
        weights['time_step'] += 0.1*exp_time_step
        exp_time_step += 1
    else:
        weights['time_step'] += 0
        exp_time_step = 1

    if energy_error > energy_error_fiducial:
        weights['energy_loss'] += 0.1*exp_energy_loss
        exp_energy_loss += 1
    else:
        weights['energy_loss'] += 0
        exp_energy_loss = 1
        """

    #print(f"Epoch [{epoch}/{num_epochs}], Train Loss: {train_loss:.4e}, Test Loss: {test_loss:.4e}, Energy Loss: {energy_error:.4e}/{energy_error_fiducial:.4e}, std: {energy_error_std:.4e}/{energy_error_fiducial_std:.4e}, Time step: {time_step:.4e}/{time_step_fiducial:.4e}")
    #print(f"Epoch [{epoch}/{num_epochs}], Train Loss: {train_res['train_loss']:.4e}, Val Loss: {val_res['val_loss']:.4e}, Energy Loss: {np.log10(train_res['energy_error']):.4e}/{np.log10(train_res['energy_error_fiducial']):.4e}, {np.log10(val_res['energy_error']):.4e}/{np.log10(val_res['energy_error_fiducial']):.4e}, Time step: {train_res['time_step']:.4e}/{train_res['time_step_fiducial']:.4e}, {val_res['time_step']:.4e}/{val_res['time_step_fiducial']:.4e}, {val_res['time_step_std']:.4e}/{val_res['time_step_fiducial_std']:.4e}")
    print(f"Epoch [{epoch}/{num_epochs}], Timestep error: {train_res['time_step_relative_error']:.4e}/{val_res['time_step_relative_error']:.4e}, Energy Loss: {np.log10(train_res['energy_error']):.4e}/{np.log10(train_res['energy_error_fiducial']):.4e}, {np.log10(val_res['energy_error']):.4e}/{np.log10(val_res['energy_error_fiducial']):.4e}, Time step: {train_res['time_step']:.4e}/{train_res['time_step_fiducial']:.4e}, {val_res['time_step']:.4e}/{val_res['time_step_fiducial']:.4e}, {val_res['time_step_std']:.4e}/{val_res['time_step_fiducial_std']:.4e}")
    #print(f"Epoch [{epoch}/{num_epochs}], Train Loss: {train_res['train_loss']:.4e}, Val Loss: {val_res['val_loss']:.4e}, Energy Loss: {np.log10(train_res['energy_error']):.4e}/{np.log10(train_res['energy_error_fiducial']):.4e}, {np.log10(val_res['energy_error']):.4e}/{np.log10(val_res['energy_error_fiducial']):.4e}, energy_pred: {energy_pred:.4e}/{energy_init:.4e}, Time step: {time_step:.4e}/{time_step_fiducial:.4e}")

[2025-04-13 20:13:40,713] [INFO] [logging.py:128:log_dist] [Rank -1] DeepSpeed info: version=0.16.4, git-hash=unknown, git-branch=unknown
[2025-04-13 20:13:40,714] [INFO] [comm.py:658:init_distributed] cdb=None
[2025-04-13 20:13:40,714] [INFO] [comm.py:673:init_distributed] Not using the DeepSpeed or dist launchers, attempting to detect MPI environment...


[2025-04-13 20:13:41,841] [INFO] [comm.py:728:mpi_discovery] Discovered MPI settings of world_rank=0, local_rank=0, world_size=1, master_addr=10.250.149.94, master_port=29500
[2025-04-13 20:13:41,842] [INFO] [comm.py:689:init_distributed] Initializing TorchBackend in DeepSpeed with backend nccl
[2025-04-13 20:13:41,845] [INFO] [config.py:734:__init__] Config mesh_device None world_size = 1


--------------------------------------------------------------------------
Ignoring value for oob_tcp_if_exclude on workergpu094 (10.250.112.0/20: Did not find interface matching this subnet).
(You can safely ignore this message.)
--------------------------------------------------------------------------


[2025-04-13 20:13:47,085] [INFO] [logging.py:128:log_dist] [Rank 0] DeepSpeed Flops Profiler Enabled: False


Using /dev/shm/.cache-yjo10/torch_extensions/py310_cu123 as PyTorch extensions root...
Creating extension directory /dev/shm/.cache-yjo10/torch_extensions/py310_cu123/fused_adam...
Detected CUDA files, patching ldflags
Emitting ninja build file /dev/shm/.cache-yjo10/torch_extensions/py310_cu123/fused_adam/build.ninja...
Building extension module fused_adam...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


### Retraining with different learning rate

In [33]:
import structures
importlib.reload(structures)
from structures import *
import losses
importlib.reload(losses)
from losses import *
import trainer
importlib.reload(trainer)
from trainer import *

ds_config = {
    "train_batch_size": batch_size,
    "gradient_accumulation_steps": 1,
    "optimizer": {
        "type": "Adam",
        "params": {"lr": 1e-5, "betas": [0.9, 0.999], "eps": 1e-8}
    },
    "fp32": {
        "enabled": True  # Enables mixed precision training
    },
    "zero_optimization": {
        "stage": 1  # Enable ZeRO Stage 1 for memory optimization
    }
}

model_engine, optimizer, _, _ = deepspeed.initialize(
    model=model,
    model_parameters=model.parameters(),
    config=ds_config
)

reset = True
if reset:
    lists = {"training_loss":[], "val_loss":[], "training_energy":[], "val_energy":[], "training_time_step":[], "val_time_step":[]}
    epoch0= 0
else:
    epoch0= epoch
# Training routine
num_epochs = 1000
for epoch in range(epoch0, num_epochs):


    #train_loss, train_energy = \
    #    train_one_epoch(model, optimizer, criterion, train_loader, input_mask, weights, device)
    train_res = train_one_epoch_deepspeed(model_engine, optimizer, criterion, train_loader, input_mask, weights, device)
        
    #test_loss, energy_error, energy_error_std, energy_error_fiducial, energy_error_fiducial_std, energy_pred, energy_init, time_step, time_step_fiducial = \
    #    validate(model, criterion, test_loader, input_mask, weights, device)
    val_res = validate_deepspeed(model_engine, criterion, test_loader, input_mask, weights, device)

    lists["training_loss"].append(train_res["train_loss"])
    lists["training_energy"].append(train_res["energy_error"])
    lists["training_time_step"].append(train_res["time_step"])

    lists["val_loss"].append(val_res["val_loss"])
    lists["val_energy"].append(val_res["energy_error"])
    lists["val_time_step"].append(val_res["time_step"])

    if epoch > 5:
        # Save the best model based on validation loss
        if val_res['val_loss'] < best_val_loss:
            best_val_loss = val_res['val_loss']
            torch.save(model.state_dict(), best_val_path)
            #traced_model = torch.jit.trace(model, example_input)
            traced_model = torch.jit.script(model)
            traced_model.save(base_path + f"c++/best_model_loss.pt")
            print(f"--> Best model saved at epoch {epoch} with val loss {best_val_loss:.4e}")
        
        # Save the best model based on validation largest timestep
        if val_res['time_step'] > best_largest_timestep:
            best_largest_timestep = val_res['time_step']
            torch.save(model.state_dict(), best_timestep_path)
            #traced_model = torch.jit.trace(model, example_input)
            traced_model = torch.jit.script(model)
            traced_model.save(base_path + f"c++/best_model_timestep.pt")
            print(f"--> Best model saved at epoch {epoch} with largest timestep {best_largest_timestep:.4e}")

    # Save the last model after every epoch (or just once at the end)
    torch.save(model.state_dict(), base_path + f"model_{epoch}.pth")
    #traced_model = torch.jit.trace(model, example_input)
    traced_model = torch.jit.script(model)
    traced_model.save(base_path + f"c++/model_{epoch}.pt")



    fig, axes = pu.generateAxesForMultiplePlots(shape=(1,3),figsize=(10,10),hspace=0.1,wspace=0.1,
                                    gridspec=None)
    for i, (key, value) in enumerate(lists.items()):
        y = i//2
        axes[y].plot(range(epoch+1), value, label=key)
        axes[y].scatter(range(epoch+1), value)
    
    axes[0].set_yscale("log")
    axes[2].set_yscale("log")

    axes[0].grid()
    axes[1].grid()
    axes[2].grid()


    axes[2].axhline(y=val_res['time_step_fiducial'], c='r',alpha=0.5,zorder=-1,label="Timestep Fiducial")
    axes[1].axhline(y=np.log10(TargetEnergyError), c='r',alpha=0.5,zorder=-1,label="Target energy")
    #plt.legend(bbox_to_anchor=(1.01, 1), loc='upper left', borderaxespad=0.)
    axes[0].legend()
    axes[1].legend()
    axes[2].legend()
    #plt.tight_layout()
    axes[0].set_ylabel("Loss")
    axes[1].set_ylabel("log(Relative Error of Energy)")
    axes[2].set_ylabel("Timestep")
    axes[2].set_xlabel("epoch")
    plt.savefig("learning_curve_re.png",dpi=100)
    plt.close()



    fig, axes = pu.generateAxesForMultiplePlots(shape=(1,2),figsize=(10,15),hspace=0.1,wspace=0.1,
                                    gridspec=None)
    # Get one batch from the validation loader
    dataiter = iter(train_loader)
    inputs, = next(dataiter)

    # Determine how many samples to take (if batch size < 50, take the whole batch)
    num_samples = min(50, batch_size)

    # Randomly select indices from the batch
    indices = random.sample(range(batch_size), num_samples)
    sampled_inputs = inputs[:,input_mask][indices]
    sampled_targets = inputs[indices,25]

    # Compute model predictions with no gradient computation
    with torch.no_grad():
        sampled_predictions = model(sampled_inputs)

    # Convert tensors to numpy arrays for plotting
    sampled_predictions = sampled_predictions[:,0].cpu().numpy().flatten()
    sampled_targets = sampled_targets.cpu().numpy().flatten()

    # Create sample indices for the x-axis (1st sample, 2nd sample, etc.)
    sample_indices = list(range(1, num_samples + 1))

    # Plot predictions and ground truth vs sample number
    # plt.figure(figsize=(8, 6))
    # Add vertical lines for each data point
    for x in sample_indices:
        axes[0].axvline(x=x, color='grey', linestyle='-', alpha=0.2,zorder=0)
    axes[0].scatter(sample_indices, sampled_targets, marker='o', label='Ground Truth', fc="none", ec="k",zorder=10)
    axes[0].scatter(sample_indices, np.power(10,sampled_predictions), marker='s', label='Predictions', fc="none",ec="r" ,zorder=10)
    axes[0].set_yscale("log")
    axes[0].set_xlabel('Sample Number')
    axes[0].set_ylabel('Value')

    # Get one batch from the validation loader
    dataiter = iter(test_loader)
    inputs, = next(dataiter)

    # Determine how many samples to take (if batch size < 50, take the whole batch)
    num_samples = min(50, batch_size)

    # Randomly select indices from the batch
    indices = random.sample(range(batch_size), num_samples)
    sampled_inputs = inputs[:,input_mask][indices]
    sampled_targets = inputs[indices,25]

    # Compute model predictions with no gradient computation
    with torch.no_grad():
        sampled_predictions = model(sampled_inputs)

    # Convert tensors to numpy arrays for plotting
    sampled_predictions = sampled_predictions[:,0].cpu().numpy().flatten()
    sampled_targets = sampled_targets.cpu().numpy().flatten()

    # Create sample indices for the x-axis (1st sample, 2nd sample, etc.)
    sample_indices = list(range(1, num_samples + 1))

    # Plot predictions and ground truth vs sample number
    # plt.figure(figsize=(8, 6))
    # Add vertical lines for each data point
    for x in sample_indices:
        axes[1].axvline(x=x, color='grey', linestyle='-', alpha=0.2,zorder=0)
    axes[1].scatter(sample_indices, sampled_targets, marker='o', label='Ground Truth', fc="none", ec="k",zorder=10)
    axes[1].scatter(sample_indices, np.power(10,sampled_predictions), marker='s', label='Predictions', fc="none",ec="r" ,zorder=10)
    axes[1].set_yscale("log")
    axes[1].set_xlabel('Sample Number')
    axes[1].set_ylabel('Value')

    axes[0].set_title(f'Epoch [{epoch}/{num_epochs}] Predictions and Ground Truth vs. Sample Number')
    #plt.legend()
    axes[0].grid()
    axes[1].grid()
    plt.legend(bbox_to_anchor=(1.01, 1), loc='upper left', borderaxespad=0.)
    plt.savefig("samples.png")
    plt.close()

    print(f"Epoch [{epoch}/{num_epochs}], Timestep error: {train_res['time_step_relative_error']:.4e}/{val_res['time_step_relative_error']:.4e}, Energy Loss: {np.log10(train_res['energy_error']):.4e}/{np.log10(train_res['energy_error_fiducial']):.4e}, {np.log10(val_res['energy_error']):.4e}/{np.log10(val_res['energy_error_fiducial']):.4e}, Time step: {train_res['time_step']:.4e}/{train_res['time_step_fiducial']:.4e}, {val_res['time_step']:.4e}/{val_res['time_step_fiducial']:.4e}, {val_res['time_step_std']:.4e}/{val_res['time_step_fiducial_std']:.4e}")

[2025-04-11 15:02:30,809] [INFO] [logging.py:128:log_dist] [Rank 0] DeepSpeed info: version=0.16.4, git-hash=unknown, git-branch=unknown
[2025-04-11 15:02:30,810] [INFO] [config.py:734:__init__] Config mesh_device None world_size = 1
[2025-04-11 15:02:30,813] [INFO] [logging.py:128:log_dist] [Rank 0] DeepSpeed Flops Profiler Enabled: False
[2025-04-11 15:02:30,814] [INFO] [logging.py:128:log_dist] [Rank 0] Using DeepSpeed Optimizer param name adam as basic optimizer
[2025-04-11 15:02:30,814] [INFO] [logging.py:128:log_dist] [Rank 0] Removing param_group that has no 'params' in the basic Optimizer
[2025-04-11 15:02:30,814] [INFO] [logging.py:128:log_dist] [Rank 0] DeepSpeed Basic Optimizer = FusedAdam
[2025-04-11 15:02:30,815] [INFO] [utils.py:59:is_zero_supported_optimizer] Checking ZeRO support for optimizer=FusedAdam type=<class 'deepspeed.ops.adam.fused_adam.FusedAdam'>
[2025-04-11 15:02:30,815] [INFO] [logging.py:128:log_dist] [Rank 0] Creating torch.float32 ZeRO stage 1 optimizer


[2025-04-11 15:02:31,347] [INFO] [utils.py:781:see_memory_usage] Before initializing optimizer states
[2025-04-11 15:02:31,351] [INFO] [utils.py:782:see_memory_usage] MA 3.75 GB         Max_MA 7.47 GB         CA 7.48 GB         Max_CA 7 GB 
[2025-04-11 15:02:31,352] [INFO] [utils.py:789:see_memory_usage] CPU Virtual Memory:  used = 58.87 GB, percent = 5.8%
[2025-04-11 15:02:31,686] [INFO] [utils.py:781:see_memory_usage] After initializing optimizer states
[2025-04-11 15:02:31,687] [INFO] [utils.py:782:see_memory_usage] MA 3.75 GB         Max_MA 3.75 GB         CA 7.48 GB         Max_CA 7 GB 
[2025-04-11 15:02:31,688] [INFO] [utils.py:789:see_memory_usage] CPU Virtual Memory:  used = 58.88 GB, percent = 5.8%
[2025-04-11 15:02:31,688] [INFO] [stage_1_and_2.py:550:__init__] optimizer state initialized
[2025-04-11 15:02:32,021] [INFO] [utils.py:781:see_memory_usage] After initializing ZeRO optimizer
[2025-04-11 15:02:32,023] [INFO] [utils.py:782:see_memory_usage] MA 3.75 GB         Max_MA 

/tmp/ipykernel_230351/2938017237.py:195: RuntimeWarning: invalid value encountered in log10
  print(f"Epoch [{epoch}/{num_epochs}], Timestep error: {train_res['time_step_relative_error']:.4e}/{val_res['time_step_relative_error']:.4e}, Energy Loss: {np.log10(train_res['energy_error']):.4e}/{np.log10(train_res['energy_error_fiducial']):.4e}, {np.log10(val_res['energy_error']):.4e}/{np.log10(val_res['energy_error_fiducial']):.4e}, Time step: {train_res['time_step']:.4e}/{train_res['time_step_fiducial']:.4e}, {val_res['time_step']:.4e}/{val_res['time_step_fiducial']:.4e}, {val_res['time_step_std']:.4e}/{val_res['time_step_fiducial_std']:.4e}")


Epoch [0/1000], Timestep error: 1.0531e+02/1.4294e+02, Energy Loss: nan/nan, nan/nan, Time step: 4.7320e-05/9.4821e-04, 4.3821e-05/8.6166e-04, 5.2429e-05/1.7175e-03
energy_error= tensor([9.7498e-03, 2.3990e-06, 4.3786e-03, 4.3996e-08, 9.7309e-05, 4.6899e-06,
        3.5696e-06, 1.8472e-02, 4.2848e-06, 1.4443e-04, 2.4365e-03, 2.8164e-06,
        1.6522e-03, 2.8938e-03, 4.1298e-03, 1.3773e-03, 8.1582e-06, 3.7944e-05,
        4.7915e-03, 1.1554e-03, 3.9258e-03, 3.4069e-04, 2.1400e-03, 1.1444e-01,
        8.4637e-03, 4.6915e-07, 1.3551e-01, 6.1601e-07, 2.5302e-06, 7.7610e+00,
        6.9206e-06, 4.0221e-03, 6.8974e-06, 7.3265e-01, 1.5260e-01, 3.8948e-02,
        4.1293e-03, 6.6361e-03, 1.0644e-04, 7.3841e-07, 6.8654e-06, 6.9131e-07,
        3.8190e-04, 1.0329e-02, 4.4091e-03, 6.2781e-07, 2.1535e-04, 4.2622e-01,
        7.0525e-06, 7.3732e-06, 5.5807e-07, 2.0005e-02, 1.2175e-03, 2.6108e-02,
        5.7828e-06, 4.2560e-05, 4.7699e-01, 1.7369e-02, 3.6069e-06, 1.0395e-07,
        7.2048e-07, 1

KeyboardInterrupt: 

## Second Gen with magnitude of velocities and acceleration (Suspended)

#### second gen is suspended because total energy is a function of each component of velocities and accelerations, unable to be written in a form of magnitudes of velocities and acceleration (it is techinically possible but not preferable neither in most cases nor in general).

In [ ]:
#from src.trainer import num_samples

# Example unsupervised data: 100 samples with 10 features each
data = np.load("../data/two_body_train_data.npy")
num_samples = data.shape[0]
num_samples_start = int(data.shape[0]*0.9)
#num_samples_start = 0
num_samples_end   = data.shape[0]
num_samples = num_samples_end - num_samples_start
num_features = data.shape[1]

# data should contain [N, pos(2) vel(2) acc(2,4) mass(1) pos(2) vel(2) acc(2,4) mass(1) energy error(1) total energy (1) dt(1)]
# new data is rearranged to [N, mag vel(1), mag acc(4), mass(1), pos(2), vel(2), acc(2,4), mass(1), pos(2), vel(2), acc(2,4), energy error(1), total energy(1), dt(1)]
data_org = torch.zeros((num_samples,num_features+5), dtype=torch.double).to(device)


# Create a tensor of features only
data_org = torch.tensor(data[num_samples_start:num_samples_end,:], dtype=torch.double).to(device)
data = data_org.clone()


# Normalize the data
data_min = data_org[:,:26].min(axis=0, keepdim=True).values
data_max = data_org[:,:26].max(axis=0, keepdim=True).values
data[:,:26] = (data_org[:,:26] - data_min) / (data_max - data_min)

# Wrap the feature tensor in a TensorDataset
# Each item from the dataset will be a tuple containing one tensor (X[i],)
dataset = TensorDataset(data)

# Define the proportion for the test set (e.g., 20%)
test_ratio = 0.2
num_test = int(num_samples * test_ratio)
num_train = num_samples - num_test

# Use random_split to split the dataset into train and test subsets
train_dataset, test_dataset = random_split(dataset, [num_train, num_test])

# Create DataLoaders for the training and testing datasets
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

### Training

In [ ]:
import structures
importlib.reload(structures)
from structures import *
import losses
importlib.reload(losses)
from losses import *
import trainer
importlib.reload(trainer)
from trainer import *


input_dim = 13     # Number of input features
hidden_dims = [64,64]     # Number of hidden neurons
output_dim = 2     # Number of output 
TargetEnergyError = 1e-5
nParticle = 2
nAttribute = 13

# Instantiate the model
try: 
    del model
    print("model deleted")
except:
    pass


#model = SimpleNN(input_dim, hidden_size, output_dim).to(device)
model = FullyConnectedNN(input_dim=input_dim, output_dim=output_dim, hidden_dims=hidden_dims, activation='relu', dropout=0.0, output_positive=True).to(device)


# Define the loss function (CrossEntropyLoss is common for classification tasks)
criterion = CustomizableLoss(nParticle=nParticle, nAttribute=nAttribute, nBatch=batch_size,alpha=0.1, beta=0.1, gamma=10.0, TargetEnergyError=TargetEnergyError,
                            data_min=data_min, data_max=data_max,device=device)

# Define the optimizer (Stochastic Gradient Descent in this example)
#optimizer = optim.SGD(model.parameters(), lr=0.001)
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Paths to save best and last models
base_path = "/mnt/home/yjo10/projects/AITimeStepper/data/models/two_body_2G/"
best_val_path = base_path + "best_model_loss.pth"
best_timestep_path = base_path + "best_model_timestep.pth"
#last_model_path = base_path + f"last_model_{epoch}.pth"
best_val_loss = float("inf")
best_largest_timestep = 0

example_input = torch.randn(1, input_dim).to(device)

# Training routine
num_epochs = 1000
for epoch in range(num_epochs):

    train_loss = \
        train_one_epoch(model, optimizer, criterion, train_loader, input_dim, device)
    test_loss, energy_error, energy_error_fiducial, time_step, time_step_fiducial = \
        validate(model, criterion, test_loader, input_dim, device)

    if epoch > 5:
        # Save the best model based on validation loss
        if test_loss < best_val_loss:
            best_val_loss = test_loss
            torch.save(model.state_dict(), best_val_path)
            traced_model = torch.jit.trace(model, example_input)
            traced_model.save(base_path + f"c++/best_model_loss.pt")
            print(f"--> Best model saved at epoch {epoch} with val loss {best_val_loss:.4e}")
        
        # Save the best model based on validation largest timestep
        if time_step > best_largest_timestep:
            best_largest_timestep = time_step
            torch.save(model.state_dict(), best_timestep_path)
            traced_model = torch.jit.trace(model, example_input)
            traced_model.save(base_path + f"c++/best_model_timestep.pt")
            print(f"--> Best model saved at epoch {epoch} with largest timestep {best_largest_timestep:.4e}")

    # Save the last model after every epoch (or just once at the end)
    torch.save(model.state_dict(), base_path + f"model_{epoch}.pth")
    traced_model = torch.jit.trace(model, example_input)
    traced_model.save(base_path + f"c++/model_{epoch}.pt")

    print(f"Epoch [{epoch}/{num_epochs}], Train Loss: {train_loss:.4e}, Test Loss: {test_loss:.4e}, Energy Loss: {energy_error:.4e}/{energy_error_fiducial:.4e}, Time step: {time_step:.4e}/{time_step_fiducial:.4e}")

# Three Body

In [6]:
#from src.trainer import num_samples

# Example unsupervised data: 100 samples with 10 features each
data = np.load("../data/three_body_train_data.npy")
num_samples = data.shape[0]
num_samples_start = int(data.shape[0]*0.9)
#num_samples_start = 0
num_samples_end   = data.shape[0]
num_samples = num_samples_end - num_samples_start
num_features = data.shape[1]

# Create a tensor of features only
data_org = torch.tensor(data[num_samples_start:num_samples_end,:], dtype=torch.double).to(device)
data = data_org.clone()



# Normalize the data
#index = np.r_[:19,20:39,40:59] # this is for three particles
#index = np.r_[:19,20:39,40:59] # this is for three particles
index = [19,39,59,60,61] # the features that must not be normalized
data_min = data_org.min(axis=0, keepdim=True).values
data_max = data_org.max(axis=0, keepdim=True).values
data_min[0,index] = 0
data_max[0,index] = 1
data = (data_org - data_min) / (data_max - data_min)

# Wrap the feature tensor in a TensorDataset
# Each item from the dataset will be a tuple containing one tensor (X[i],)
dataset = TensorDataset(data)

# Define the proportion for the test set (e.g., 20%)
test_ratio = 0.2
num_test = int(num_samples * test_ratio)
num_train = num_samples - num_test

# Use random_split to split the dataset into train and test subsets
train_dataset, test_dataset = random_split(dataset, [num_train, num_test])

# Create DataLoaders for the training and testing datasets
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
print(data.shape)

torch.Size([7793, 62])


### Loss function test


In [22]:
import structures
importlib.reload(structures)
from structures import *
import losses
importlib.reload(losses)
from losses import *
TargetEnergyError = 1e-5
batch_size = 32
criterion = CustomizableLoss3D(nParticle=3, nAttribute=20, nBatch=batch_size,alpha=0.1, beta=0.1, gamma=10.0, TargetEnergyError=TargetEnergyError,
                            data_min=data_min, data_max=data_max,device=device)
X = data[0:10,:]
output = torch.zeros((X.shape[0],2)).to(device)
loss, loss_terms = criterion(output, X)
X[:,-2], X[:,-1]

kin= tensor([7.3576e+13, 7.3576e+13, 7.3576e+13, 3.9447e+13, 3.9447e+13, 3.9447e+13,
        6.7117e+13, 6.7117e+13, 6.7117e+13, 3.4046e+14], device='cuda:0',
       grad_fn=<MulBackward0>)
pot= tensor([-3.0354e+15, -3.0354e+15, -3.0354e+15, -3.0015e+15, -3.0015e+15,
        -3.0015e+15, -3.0289e+15, -3.0289e+15, -3.0289e+15, -3.3026e+15],
       device='cuda:0', grad_fn=<SubBackward0>)
init0= tensor([-2.9619e+15, -2.9619e+15, -2.9619e+15, -2.9620e+15, -2.9620e+15,
        -2.9620e+15, -2.9618e+15, -2.9618e+15, -2.9618e+15, -2.9622e+15],
       device='cuda:0') 
init1= tensor([-2.9619e+15, -2.9619e+15, -2.9619e+15, -2.9620e+15, -2.9620e+15,
        -2.9620e+15, -2.9618e+15, -2.9618e+15, -2.9618e+15, -2.9622e+15],
       device='cuda:0', grad_fn=<AddBackward0>)
diff= tensor([-0.5000, -0.5000, -0.5000,  0.5000,  0.5000,  0.5000,  0.5000,  1.0000,
         0.5000, -1.5000], device='cuda:0', grad_fn=<SubBackward0>)
kin= tensor([ 348016.2793,  348016.2793,  348016.2793,  206479.9556,  20647

(tensor([6.2572e-05, 6.2572e-05, 6.2572e-05, 9.6054e-05, 9.6054e-05, 9.6054e-05,
         1.4517e-04, 1.4517e-04, 1.4517e-04, 4.0750e-05], device='cuda:0'),
 tensor([-2.9619e+15, -2.9619e+15, -2.9619e+15, -2.9620e+15, -2.9620e+15,
         -2.9620e+15, -2.9618e+15, -2.9618e+15, -2.9618e+15, -2.9622e+15],
        device='cuda:0'))

### Training

In [10]:
import structures
importlib.reload(structures)
from structures import *
import losses
importlib.reload(losses)
from losses import *
import trainer
importlib.reload(trainer)
from trainer import *


input_size = 16     # Number of input features
input_mask = np.r_[0,4:19]
hidden_dims = [32,64,32]     # Number of hidden neurons
output_size = 2     # Number of output 
TargetEnergyError = 1e-5


# Instantiate the model
try: 
    del model
    print("model deleted")
except:
    pass

#model = SimpleNN(input_size, hidden_size, output_size).to(device)
model = FullyConnectedNN(input_dim=input_size, output_dim=output_size, hidden_dims=hidden_dims, activation='relu', dropout=0.0, output_positive=True).to(device)


# Define the loss function (CrossEntropyLoss is common for classification tasks)
criterion = CustomizableLoss3D(nParticle=2, nAttribute=20, nBatch=batch_size,alpha=0.1, beta=1.0, gamma=1.0, TargetEnergyError=TargetEnergyError,
                            data_min=data_min, data_max=data_max,device=device)

# Define the optimizer (Stochastic Gradient Descent in this example)
#optimizer = optim.SGD(model.parameters(), lr=0.001)
#optimizer = optim.Adam(model.parameters(), lr=0.01)
optimizer = optim.AdamW(model.parameters(), lr=0.01)



# Paths to save best and last models
base_path = "/mnt/home/yjo10/projects/AITimeStepper/data/models/three_body/"
best_val_path = base_path + "best_model_loss.pth"
best_timestep_path = base_path + "best_model_timestep.pth"
#last_model_path = base_path + f"last_model_{epoch}.pth"
best_val_loss = float("inf")
best_largest_timestep = 0

example_input = torch.randn(1, input_size).to(device)

# Training routine
num_epochs = 1000
for epoch in range(num_epochs):

    train_loss = \
        train_one_epoch(model, optimizer, criterion, train_loader, input_mask, device)
    test_loss, energy_error, energy_error_std, energy_error_fiducial, energy_error_fiducial_std, energy_pred, energy_init, time_step, time_step_fiducial = \
        validate(model, criterion, test_loader, input_mask, device)

    if epoch > 5:
        # Save the best model based on validation loss
        if test_loss < best_val_loss:
            best_val_loss = test_loss
            torch.save(model.state_dict(), best_val_path)
            traced_model = torch.jit.trace(model, example_input)
            traced_model.save(base_path + f"c++/best_model_loss.pt")
            print(f"--> Best model saved at epoch {epoch} with val loss {best_val_loss:.4e}")
        
        # Save the best model based on validation largest timestep
        if time_step > best_largest_timestep:
            best_largest_timestep = time_step
            torch.save(model.state_dict(), best_timestep_path)
            traced_model = torch.jit.trace(model, example_input)
            traced_model.save(base_path + f"c++/best_model_timestep.pt")
            print(f"--> Best model saved at epoch {epoch} with largest timestep {best_largest_timestep:.4e}")

    # Save the last model after every epoch (or just once at the end)
    torch.save(model.state_dict(), base_path + f"model_{epoch}.pth")
    traced_model = torch.jit.trace(model, example_input)
    traced_model.save(base_path + f"c++/model_{epoch}.pt")

    #print(f"Epoch [{epoch}/{num_epochs}], Train Loss: {train_loss:.4e}, Test Loss: {test_loss:.4e}, Energy Loss: {energy_error:.4e}/{energy_error_fiducial:.4e}, std: {energy_error_std:.4e}/{energy_error_fiducial_std:.4e}, Time step: {time_step:.4e}/{time_step_fiducial:.4e}")
    print(f"Epoch [{epoch}/{num_epochs}], Train Loss: {train_loss:.4e}, Test Loss: {test_loss:.4e}, Energy Loss: {energy_error:.4e}/{energy_error_fiducial:.4e}, energy_pred: {energy_pred:.4e}/{energy_init:.4e}, Time step: {time_step:.4e}/{time_step_fiducial:.4e}")

model deleted
Epoch [0/1000], Train Loss: 2.0895e+120, Test Loss: 7.8070e+77, Energy Loss: 7.0442e+21/1.8740e-01, energy_pred: -1.6958e+38/-3.3775e+16, Time step: 4.2222e-04/5.6198e-08
Epoch [1/1000], Train Loss: 8.8817e+78, Test Loss: 3.9814e+79, Energy Loss: 5.1009e+22/1.8740e-01, energy_pred: -1.2275e+39/-3.3775e+16, Time step: 5.6410e-04/5.6198e-08
Epoch [2/1000], Train Loss: 3.5948e+80, Test Loss: 1.7811e+81, Energy Loss: 3.4587e+23/1.8740e-01, energy_pred: -8.3154e+39/-3.3775e+16, Time step: 7.4651e-04/5.6198e-08
Epoch [3/1000], Train Loss: 1.8261e+82, Test Loss: 6.7488e+82, Energy Loss: 2.1677e+24/1.8740e-01, energy_pred: -5.1995e+40/-3.3775e+16, Time step: 9.7750e-04/5.6198e-08
Epoch [4/1000], Train Loss: 5.7342e+83, Test Loss: 2.1832e+84, Energy Loss: 1.2611e+25/1.8740e-01, energy_pred: -3.0146e+41/-3.3775e+16, Time step: 1.2670e-03/5.6198e-08
Epoch [5/1000], Train Loss: 1.6567e+85, Test Loss: 6.2409e+85, Energy Loss: 6.8746e+25/1.8740e-01, energy_pred: -1.6372e+42/-3.3775e+16

KeyboardInterrupt: 

### Re-training

In [ ]:
optimizer = optim.AdamW(model.parameters(), lr=0.001)

# Training routine
num_epochs = 1000
epoch0= epoch
for epoch in range(epoch0, num_epochs):

    train_loss = \
        train_one_epoch(model, optimizer, criterion, train_loader, input_mask, device)
    test_loss, energy_error, energy_error_std, energy_error_fiducial, energy_error_fiducial_std, time_step, time_step_fiducial = \
        validate(model, criterion, test_loader, input_mask, device)

    if epoch > 5:
        # Save the best model based on validation loss
        if test_loss < best_val_loss:
            best_val_loss = test_loss
            torch.save(model.state_dict(), best_val_path)
            traced_model = torch.jit.trace(model, example_input)
            traced_model.save(base_path + f"c++/best_model_loss.pt")
            print(f"--> Best model saved at epoch {epoch} with val loss {best_val_loss:.4e}")
        
        # Save the best model based on validation largest timestep
        if time_step > best_largest_timestep:
            best_largest_timestep = time_step
            torch.save(model.state_dict(), best_timestep_path)
            traced_model = torch.jit.trace(model, example_input)
            traced_model.save(base_path + f"c++/best_model_timestep.pt")
            print(f"--> Best model saved at epoch {epoch} with largest timestep {best_largest_timestep:.4e}")

    # Save the last model after every epoch (or just once at the end)
    torch.save(model.state_dict(), base_path + f"model_{epoch}.pth")
    traced_model = torch.jit.trace(model, example_input)
    traced_model.save(base_path + f"c++/model_{epoch}.pt")

    print(f"Epoch [{epoch}/{num_epochs}], Train Loss: {train_loss:.4e}, Test Loss: {test_loss:.4e}, Energy Loss: {energy_error:.4e}/{energy_error_fiducial:.4e}, std: {energy_error_std:.4e}/{energy_error_fiducial_std:.4e}, Time step: {time_step:.4e}/{time_step_fiducial:.4e}")

### loading models

In [ ]:
input_size = 16     # Number of input features
input_mask = np.r_[0,4:19]
hidden_dims = [64,64]     # Number of hidden neurons
output_size = 2     # Number of output 
TargetEnergyError = 1e-5

base_path = "/mnt/home/yjo10/projects/AITimeStepper/data/models/three_body/"
best_val_path = base_path + "best_model_loss.pth"
best_timestep_path = base_path + "best_model_timestep.pth"
epoch = 73
model_path = f"model_{epoch}.pth"

# Instantiate the model
model = FullyConnectedNN(input_dim=input_size, output_dim=output_size, hidden_dims=hidden_dims, activation='relu', dropout=0.1, output_positive=True).to(device)

# Load the state dictionary from a file
state_dict = torch.load(base_path + model_path, map_location=torch.device('cpu'))
model.load_state_dict(state_dict)

# Set the model to evaluation mode (important for inference)
model.eval()

# Define the loss function (CrossEntropyLoss is common for classification tasks)
criterion = CustomizableLoss3D(nParticle=2, nAttribute=20, nBatch=batch_size,alpha=0.1, beta=1.0, gamma=1.0, TargetEnergyError=TargetEnergyError,
                            data_min=data_min, data_max=data_max,device=device)


test_loss, energy_error, energy_error_std, energy_error_fiducial, energy_error_fiducial_std, time_step, time_step_fiducial = \
    validate(model, criterion, test_loader, input_mask, device)
print(f"Epoch [{epoch}], Test Loss: {test_loss:.4e}, Energy Loss: {energy_error:.4e}/{energy_error_fiducial:.4e}, std: {energy_error_std:.4e}/{energy_error_fiducial_std:.4e}, Time step: {time_step:.4e}/{time_step_fiducial:.4e}")

### Optimizing Training

# Three Body Only With Magnitude

In [36]:
def normalizer(_data):
    _max = torch.abs(_data.max())
    data_max = torch.ones(_data.size(), device=device)*_max
    _data = _data/data_max
    return _data, _max
    
def denormalizer(_data, _max):
    data_max = torch.ones(_data.size(), device=device)*_max
    return _data*data_max

# Example unsupervised data: 100 samples with 10 features each
data = np.load("../data/three_body_train_data.npy")
num_samples = data.shape[0]
num_samples_start = int(data.shape[0]*0.8)
num_samples_start = 0
num_samples_end   = data.shape[0]
num_samples = num_samples_end - num_samples_start
num_features = data.shape[1]

# Placeholder for input (magnitudes of velocities and accelerations)
data_org = torch.tensor(data[num_samples_start:num_samples_end,:], dtype=dtype).to(device)
data = torch.empty((num_samples,6+num_features), dtype=dtype).to(device)
data[:,6:] = data_org


# Magnitudes vel, acc, mass
temp, tmax = normalizer(data_org[:,4:7])
vel = denormalizer(torch.norm(temp, p=2, dim=1),tmax)
acc = torch.empty((num_samples,4), dtype=dtype).to(device)

temp,tmax = normalizer(data_org[:,7:10])
acc[:,0] = denormalizer(torch.norm(temp, p=2, dim=1),tmax)
temp,tmax = normalizer(data_org[:,10:13])
acc[:,1] = denormalizer(torch.norm(temp, p=2, dim=1),tmax)
temp,tmax = normalizer(data_org[:,13:16])
acc[:,2] = denormalizer(torch.norm(temp, p=2, dim=1),tmax)
temp,tmax = normalizer(data_org[:,16:19])
acc[:,3] = denormalizer(torch.norm(temp, p=2, dim=1),tmax)
mass = data_org[:,0]

data[:,0] = vel
data[:,1:5] = acc
data[:,5] = mass

# Normalize the data
eps = 1e-5
data_min = data[:,:6].min(axis=0, keepdim=True).values
data_max = data[:,:6].max(axis=0, keepdim=True).values
data[:,:6] = (data[:,:6] - data_min) / (data_max - data_min)+eps


# Wrap the feature tensor in a TensorDataset
# Each item from the dataset will be a tuple containing one tensor (X[i],)
dataset = TensorDataset(data)

# Define the proportion for the test set (e.g., 20%)
test_ratio = 0.2
num_test = int(num_samples * test_ratio)
num_train = num_samples - num_test

# Use random_split to split the dataset into train and test subsets
# Define generator with a fixed seed
generator = torch.Generator().manual_seed(42)
train_dataset, test_dataset = random_split(dataset, [num_train, num_test], generator=generator)

# Create DataLoaders for the training and testing datasets
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
print(data.shape)

torch.Size([17694, 68])


### Loss function test


In [69]:
import structures
importlib.reload(structures)
from structures import *
import losses
importlib.reload(losses)
from losses import *
TargetEnergyError = 1e-5
#batch_size = 32
criterion = CustomizableLoss3DM(nParticle=3, nAttribute=20, nBatch=batch_size,alpha=0.1, beta=0.1, gamma=10.0, TargetEnergyError=TargetEnergyError,
                            data_min=data_min, data_max=data_max,device=device)
X = data[:20,:]
output = data[:20,[25,25]] #torch.zeros((X.shape[0],2)).to(device)
loss, loss_terms = criterion(output, X)
#output[:,0], X[:,-2], X[:,-1]

1e-05
tensor([3.2997e-04, 2.5714e+04, 3.4986e-04, 3.4986e-04, 8.9298e+04, 3.8232e-03,
        3.8232e-03, 3.6064e+05, 1.2763e-02, 1.2763e-02, 2.1405e+06, 6.9280e-03,
        6.9280e-03, 7.2961e+06, 2.3218e-02, 2.3218e-02, 3.8597e+07, 5.3466e-03,
        5.3466e-03, 1.1962e+08], device='cuda:0', grad_fn=<AbsBackward0>)


In [31]:
data[:,25]#.mean()

tensor([6.2500e-06, 6.2500e-06, 6.2500e-06,  ..., 1.2500e-05, 2.4410e-08,
        2.4410e-08], device='cuda:0')

In [61]:
data[train_dataset.indices,:3].min(dim=0), data[train_dataset.indices,:3].max(dim=0), data[train_dataset.indices,:3].mean(dim=0)

(torch.return_types.min(
 values=tensor([0., 0., 0.], device='cuda:0'),
 indices=tensor([5041,    9,    9], device='cuda:0')),
 torch.return_types.max(
 values=tensor([1., 1., 1.], device='cuda:0'),
 indices=tensor([1649, 1649, 1649], device='cuda:0')),
 tensor([0.2625, 0.1272, 0.0918], device='cuda:0'))

In [62]:
data[test_dataset.indices,:3].min(dim=0), data[test_dataset.indices,:3].max(dim=0), data[test_dataset.indices,:3].mean(dim=0)

(torch.return_types.min(
 values=tensor([0.0003, 0.0000, 0.0000], device='cuda:0'),
 indices=tensor([368,  18,  18], device='cuda:0')),
 torch.return_types.max(
 values=tensor([0.9988, 0.9888, 0.9852], device='cuda:0'),
 indices=tensor([2865, 2865, 2865], device='cuda:0')),
 tensor([0.2565, 0.1216, 0.0869], device='cuda:0'))

In [ ]:
train_loader.

### Training

In [37]:
import structures
importlib.reload(structures)
from structures import *
import losses
importlib.reload(losses)
from losses import *
import trainer
importlib.reload(trainer)
from trainer import *

input_mask = np.r_[0:6]
input_size = len(input_mask)
hidden_dims = [32,64,64,32]     # Number of hidden neurons
#hidden_dims = [8,16,16,4]     # Number of hidden neurons
#hidden_dims = [16,128,32,8]     # Number of hidden neurons
#hidden_dims = [8,8]     # Number of hidden neurons
output_size = 2     # Number of output 
TargetEnergyError = 1e-2
weights = {"time_step":1, "energy_loss":1}

# Instantiate the model
try: 
    del mode
    l
    print("model deleted")
except:
    pass


ds_config = {
    "train_batch_size": batch_size,
    "gradient_accumulation_steps": 1,
    "optimizer": {
        "type": "Adam",
        "params": {"lr": 1e-4, "betas": [0.9, 0.999], "eps": 1e-8}
    },
    "fp32": {
        "enabled": True  # Enables mixed precision training
    },
    "zero_optimization": {
        "stage": 1  # Enable ZeRO Stage 1 for memory optimization
    }
}


#model = SimpleNN(input_size, hidden_size, output_size).to(device)
model = FullyConnectedNN(input_dim=input_size, output_dim=output_size, hidden_dims=hidden_dims, activation='relu', dropout=0.0, output_positive=True).to(device)


model_engine, optimizer, _, _ = deepspeed.initialize(
    model=model,
    model_parameters=model.parameters(),
    config=ds_config
)

# Define the loss function (CrossEntropyLoss is common for classification tasks)
criterion = CustomizableLoss3DM(nParticle=3, nAttribute=20, nBatch=batch_size,alpha=weights['time_step'], beta=weights['energy_loss'], gamma=weights['energy_loss'], TargetEnergyError=TargetEnergyError,
                            data_min=data_min, data_max=data_max,device=device)

# Define the optimizer (Stochastic Gradient Descent in this example)
#optimizer = optim.SGD(model.parameters(), lr=0.001)
#optimizer = optim.Adam(model.parameters(), lr=1e-5)
#optimizer = optim.AdamW(model.parameters(), lr=0.00001)
#optimizer = optim.AdamW(model.parameters(), lr=0.00001)



# Paths to save best and last models
base_path = "/mnt/home/yjo10/projects/AITimeStepper/data/models/three_body/"
best_val_path = base_path + "best_model_loss.pth"
best_timestep_path = base_path + "best_model_timestep.pth"
#last_model_path = base_path + f"last_model_{epoch}.pth"
best_val_loss = float("inf")
best_largest_timestep = 0
with open(base_path+"c++/normalization_factors.txt", "w") as f:
    f.write(f"{data_min[0][0]} {data_min[0][1]} {data_min[0][2]} {data_min[0][3]} {data_min[0][4]} {data_min[0][5]}\n")    
    f.write(f"{data_max[0][0]} {data_max[0][1]} {data_max[0][2]} {data_max[0][3]} {data_max[0][4]} {data_max[0][5]}\n")    

example_input = torch.randn(1, input_size).to(device)
exp_time_step = 1
exp_energy_loss = 1
# Training routine
num_epochs = 1000
lists = {"training_loss":[], "val_loss":[], "training_energy":[], "val_energy":[], "training_time_step":[], "val_time_step":[]}

for epoch in range(num_epochs):

    #train_loss, train_energy = \
    #    train_one_epoch(model, optimizer, criterion, train_loader, input_mask, weights, device)
    train_res = train_one_epoch_deepspeed(model_engine, optimizer, criterion, train_loader, input_mask, weights, device)
        
    #test_loss, energy_error, energy_error_std, energy_error_fiducial, energy_error_fiducial_std, energy_pred, energy_init, time_step, time_step_fiducial = \
    #    validate(model, criterion, test_loader, input_mask, weights, device)
    val_res = validate_deepspeed(model_engine, criterion, test_loader, input_mask, weights, device)

    lists["training_loss"].append(train_res["train_loss"])
    lists["training_energy"].append(train_res["energy_error"])
    lists["training_time_step"].append(train_res["time_step"])

    lists["val_loss"].append(val_res["val_loss"])
    lists["val_energy"].append(val_res["energy_error"])
    lists["val_time_step"].append(val_res["time_step"])

    if epoch > 5:
        # Save the best model based on validation loss
        if val_res['val_loss'] < best_val_loss:
            best_val_loss = val_res['val_loss']
            torch.save(model.state_dict(), best_val_path)
            #traced_model = torch.jit.trace(model, example_input)
            traced_model = torch.jit.script(model)
            traced_model.save(base_path + f"c++/best_model_loss.pt")
            print(f"--> Best model saved at epoch {epoch} with val loss {best_val_loss:.4e}")
        
        # Save the best model based on validation largest timestep
        if val_res['time_step'] > best_largest_timestep:
            best_largest_timestep = val_res['time_step']
            torch.save(model.state_dict(), best_timestep_path)
            #traced_model = torch.jit.trace(model, example_input)
            traced_model = torch.jit.script(model)
            traced_model.save(base_path + f"c++/best_model_timestep.pt")
            print(f"--> Best model saved at epoch {epoch} with largest timestep {best_largest_timestep:.4e}")

    # Save the last model after every epoch (or just once at the end)
    torch.save(model.state_dict(), base_path + f"model_{epoch}.pth")
    #traced_model = torch.jit.trace(model, example_input)
    traced_model = torch.jit.script(model)
    traced_model.save(base_path + f"c++/model_{epoch}.pt")



    fig, axes = pu.generateAxesForMultiplePlots(shape=(1,3),figsize=(10,10),hspace=0.1,wspace=0.1,
                                    gridspec=None)
    for i, (key, value) in enumerate(lists.items()):
        y = i//2
        axes[y].plot(range(epoch+1), value, label=key)
        axes[y].scatter(range(epoch+1), value)
        axes[y].set_yscale("log")


    axes[2].axhline(y=val_res['time_step_fiducial'], c='r',alpha=0.5,zorder=-1,label="Timestep Fiducial")
    axes[1].axhline(y=TargetEnergyError, c='r',alpha=0.5,zorder=-1,label="Target energy")
    #plt.legend(bbox_to_anchor=(1.01, 1), loc='upper left', borderaxespad=0.)
    axes[0].legend()
    axes[1].legend()
    axes[2].legend()
    #plt.tight_layout()
    plt.savefig("learning_curve.png",dpi=100)
    plt.close()



    fig, axes = pu.generateAxesForMultiplePlots(shape=(1,2),figsize=(10,15),hspace=0.1,wspace=0.1,
                                    gridspec=None)
    # Get one batch from the validation loader
    dataiter = iter(train_loader)
    inputs, = next(dataiter)

    # Determine how many samples to take (if batch size < 50, take the whole batch)
    num_samples = min(50, batch_size)

    # Randomly select indices from the batch
    indices = random.sample(range(batch_size), num_samples)
    sampled_inputs = inputs[:,input_mask][indices]
    sampled_targets = inputs[indices,25]

    # Compute model predictions with no gradient computation
    with torch.no_grad():
        sampled_predictions = model(sampled_inputs)

    # Convert tensors to numpy arrays for plotting
    sampled_predictions = sampled_predictions[:,0].cpu().numpy().flatten()
    sampled_targets = sampled_targets.cpu().numpy().flatten()

    # Create sample indices for the x-axis (1st sample, 2nd sample, etc.)
    sample_indices = list(range(1, num_samples + 1))

    # Plot predictions and ground truth vs sample number
    # plt.figure(figsize=(8, 6))
    # Add vertical lines for each data point
    for x in sample_indices:
        axes[0].axvline(x=x, color='grey', linestyle='-', alpha=0.2,zorder=0)
    axes[0].scatter(sample_indices, sampled_targets, marker='o', label='Ground Truth', fc="none", ec="k",zorder=10)
    axes[0].scatter(sample_indices, sampled_predictions, marker='s', label='Predictions', fc="none",ec="r" ,zorder=10)
    axes[0].set_yscale("log")
    axes[0].set_xlabel('Sample Number')
    axes[0].set_ylabel('Value')

    # Get one batch from the validation loader
    dataiter = iter(test_loader)
    inputs, = next(dataiter)

    # Determine how many samples to take (if batch size < 50, take the whole batch)
    num_samples = min(50, batch_size)

    # Randomly select indices from the batch
    indices = random.sample(range(batch_size), num_samples)
    sampled_inputs = inputs[:,input_mask][indices]
    sampled_targets = inputs[indices,25]

    # Compute model predictions with no gradient computation
    with torch.no_grad():
        sampled_predictions = model(sampled_inputs)

    # Convert tensors to numpy arrays for plotting
    sampled_predictions = sampled_predictions[:,0].cpu().numpy().flatten()
    sampled_targets = sampled_targets.cpu().numpy().flatten()

    # Create sample indices for the x-axis (1st sample, 2nd sample, etc.)
    sample_indices = list(range(1, num_samples + 1))

    # Plot predictions and ground truth vs sample number
    # plt.figure(figsize=(8, 6))
    # Add vertical lines for each data point
    for x in sample_indices:
        axes[1].axvline(x=x, color='grey', linestyle='-', alpha=0.2,zorder=0)
    axes[1].scatter(sample_indices, sampled_targets, marker='o', label='Ground Truth', fc="none", ec="k",zorder=10)
    axes[1].scatter(sample_indices, sampled_predictions, marker='s', label='Predictions', fc="none",ec="r" ,zorder=10)
    axes[1].set_yscale("log")
    axes[1].set_xlabel('Sample Number')
    axes[1].set_ylabel('Value')

    axes[0].set_title(f'Epoch [{epoch}/{num_epochs}] Predictions and Ground Truth vs. Sample Number')
    #plt.legend()
    plt.legend(bbox_to_anchor=(1.01, 1), loc='upper left', borderaxespad=0.)
    plt.savefig("samples.png")
    plt.close()


    """
    if time_step < time_step_fiducial:
        weights['time_step'] += 0.1*exp_time_step
        exp_time_step += 1
    else:
        weights['time_step'] += 0
        exp_time_step = 1

    if energy_error > energy_error_fiducial:
        weights['energy_loss'] += 0.1*exp_energy_loss
        exp_energy_loss += 1
    else:
        weights['energy_loss'] += 0
        exp_energy_loss = 1
        """

    #print(f"Epoch [{epoch}/{num_epochs}], Train Loss: {train_loss:.4e}, Test Loss: {test_loss:.4e}, Energy Loss: {energy_error:.4e}/{energy_error_fiducial:.4e}, std: {energy_error_std:.4e}/{energy_error_fiducial_std:.4e}, Time step: {time_step:.4e}/{time_step_fiducial:.4e}")
    #print(f"Epoch [{epoch}/{num_epochs}], Train Loss: {train_res['train_loss']:.4e}, Val Loss: {val_res['val_loss']:.4e}, Energy Loss: {np.log10(train_res['energy_error']):.4e}/{np.log10(train_res['energy_error_fiducial']):.4e}, {np.log10(val_res['energy_error']):.4e}/{np.log10(val_res['energy_error_fiducial']):.4e}, Time step: {train_res['time_step']:.4e}/{train_res['time_step_fiducial']:.4e}, {val_res['time_step']:.4e}/{val_res['time_step_fiducial']:.4e}, {val_res['time_step_std']:.4e}/{val_res['time_step_fiducial_std']:.4e}")
    print(f"Epoch [{epoch}/{num_epochs}], Timestep error: {train_res['time_step_relative_error']:.4e}/{val_res['time_step_relative_error']:.4e}, Energy Loss: {np.log10(train_res['energy_error']):.4e}/{np.log10(train_res['energy_error_fiducial']):.4e}, {np.log10(val_res['energy_error']):.4e}/{np.log10(val_res['energy_error_fiducial']):.4e}, Time step: {train_res['time_step']:.4e}/{train_res['time_step_fiducial']:.4e}, {val_res['time_step']:.4e}/{val_res['time_step_fiducial']:.4e}, {val_res['time_step_std']:.4e}/{val_res['time_step_fiducial_std']:.4e}")
    #print(f"Epoch [{epoch}/{num_epochs}], Train Loss: {train_res['train_loss']:.4e}, Val Loss: {val_res['val_loss']:.4e}, Energy Loss: {np.log10(train_res['energy_error']):.4e}/{np.log10(train_res['energy_error_fiducial']):.4e}, {np.log10(val_res['energy_error']):.4e}/{np.log10(val_res['energy_error_fiducial']):.4e}, energy_pred: {energy_pred:.4e}/{energy_init:.4e}, Time step: {time_step:.4e}/{time_step_fiducial:.4e}")

[2025-04-06 23:38:28,868] [INFO] [logging.py:128:log_dist] [Rank 0] DeepSpeed info: version=0.16.4, git-hash=unknown, git-branch=unknown
[2025-04-06 23:38:28,869] [INFO] [config.py:734:__init__] Config mesh_device None world_size = 1
[2025-04-06 23:38:28,873] [INFO] [logging.py:128:log_dist] [Rank 0] DeepSpeed Flops Profiler Enabled: False
[2025-04-06 23:38:28,874] [INFO] [logging.py:128:log_dist] [Rank 0] Using DeepSpeed Optimizer param name adam as basic optimizer
[2025-04-06 23:38:28,875] [INFO] [logging.py:128:log_dist] [Rank 0] Removing param_group that has no 'params' in the basic Optimizer
[2025-04-06 23:38:28,875] [INFO] [logging.py:128:log_dist] [Rank 0] DeepSpeed Basic Optimizer = FusedAdam
[2025-04-06 23:38:28,876] [INFO] [utils.py:59:is_zero_supported_optimizer] Checking ZeRO support for optimizer=FusedAdam type=<class 'deepspeed.ops.adam.fused_adam.FusedAdam'>
[2025-04-06 23:38:28,877] [INFO] [logging.py:128:log_dist] [Rank 0] Creating torch.float32 ZeRO stage 1 optimizer


KeyboardInterrupt: 

### Re-training

In [ ]:
optimizer = optim.AdamW(model.parameters(), lr=0.001)

# Training routine
num_epochs = 1000
epoch0= epoch
for epoch in range(epoch0, num_epochs):

    train_loss = \
        train_one_epoch(model, optimizer, criterion, train_loader, input_mask, weights, device)
    test_loss, energy_error, energy_error_std, energy_error_fiducial, energy_error_fiducial_std, energy_pred, energy_init, time_step, time_step_fiducial = \
        validate(model, criterion, test_loader, input_mask, weights, device)

    if epoch > 5:
        # Save the best model based on validation loss
        if test_loss < best_val_loss:
            best_val_loss = test_loss
            torch.save(model.state_dict(), best_val_path)
            traced_model = torch.jit.trace(model, example_input)
            traced_model.save(base_path + f"c++/best_model_loss.pt")
            print(f"--> Best model saved at epoch {epoch} with val loss {best_val_loss:.4e}")
        
        # Save the best model based on validation largest timestep
        if time_step > best_largest_timestep:
            best_largest_timestep = time_step
            torch.save(model.state_dict(), best_timestep_path)
            traced_model = torch.jit.trace(model, example_input)
            traced_model.save(base_path + f"c++/best_model_timestep.pt")
            print(f"--> Best model saved at epoch {epoch} with largest timestep {best_largest_timestep:.4e}")

    # Save the last model after every epoch (or just once at the end)
    torch.save(model.state_dict(), base_path + f"model_{epoch}.pth")
    traced_model = torch.jit.trace(model, example_input)
    traced_model.save(base_path + f"c++/model_{epoch}.pt")

    #print(f"Epoch [{epoch}/{num_epochs}], Train Loss: {train_loss:.4e}, Test Loss: {test_loss:.4e}, Energy Loss: {energy_error:.4e}/{energy_error_fiducial:.4e}, std: {energy_error_std:.4e}/{energy_error_fiducial_std:.4e}, Time step: {time_step:.4e}/{time_step_fiducial:.4e}")
    print(f"Epoch [{epoch}/{num_epochs}], Train Loss: {train_loss:.4e}, Test Loss: {test_loss:.4e}, Energy Loss: {np.log10(energy_error):.4e}/{np.log10(energy_error_fiducial):.4e}, energy_pred: {energy_pred:.4e}/{energy_init:.4e}, Time step: {time_step:.4e}/{time_step_fiducial:.4e}")

### loading models

In [7]:
input_size = 6     # Number of input features
input_mask = np.r_[0:6]
hidden_dims = [64,64]     # Number of hidden neurons
hidden_dims = [16,32,16]     # Number of hidden neurons
output_size = 2     # Number of output 
TargetEnergyError = 1e-5

base_path = "/mnt/home/yjo10/projects/AITimeStepper/data/models/three_body/"
best_val_path = base_path + "best_model_loss.pth"
best_timestep_path = base_path + "best_model_timestep.pth"
epoch = 13
epoch = 20


model_path = f"model_{epoch}.pth"

weights = {"time_step":5, "energy_loss":1e-1}

# Instantiate the model
model = FullyConnectedNN(input_dim=input_size, output_dim=output_size, hidden_dims=hidden_dims, activation='relu', dropout=0.1, output_positive=True).to(device)

# Load the state dictionary from a file
state_dict = torch.load(base_path + model_path, map_location=torch.device('cpu'))
model.load_state_dict(state_dict)

# Set the model to evaluation mode (important for inference)
model.eval()

# Define the loss function (CrossEntropyLoss is common for classification tasks)
criterion = CustomizableLoss3DM(nParticle=3, nAttribute=20, nBatch=batch_size,alpha=0.1, beta=1.0, gamma=1.0, TargetEnergyError=TargetEnergyError,
                            data_min=data_min, data_max=data_max,device=device)


test_loss, energy_error, energy_error_std, energy_error_fiducial, energy_error_fiducial_std, energy_pred, energy_init, time_step, time_step_fiducial = \
    validate(model, criterion, test_loader, input_mask, weights, device)
print(f"Epoch [{epoch}], Test Loss: {test_loss:.4e}, Energy Loss: {energy_error:.4e}/{energy_error_fiducial:.4e}, std: {energy_error_std:.4e}/{energy_error_fiducial_std:.4e}, Time step: {time_step:.4e}/{time_step_fiducial:.4e}")

Epoch [20], Test Loss: 4.4118e+01, Energy Loss: 1.3661e-03/2.7618e-01, std: 1.0723e-03/1.8289e-01, Time step: 7.1532e-11/2.0075e-08


### Optimizing Training

# Three Body from Canonical Timestepper

### dataload

In [2]:
def normalizer(_data):
    _max = torch.abs(_data.max())
    data_max = torch.ones(_data.size(), device=device)*_max
    _data = _data/data_max
    return _data, _max
    
def denormalizer(_data, _max):
    data_max = torch.ones(_data.size(), device=device)*_max
    return _data*data_max

# Example unsupervised data: 100 samples with 10 features each
data = np.load("../data/three_body_train_data.npy")
num_samples = data.shape[0]
num_samples_start = int(data.shape[0]*0.95)
num_samples_start = 0
num_samples_end   = data.shape[0]
num_samples = num_samples_end - num_samples_start
num_features = data.shape[1]

# Placeholder for input (magnitudes of velocities and accelerations)
data_org = torch.tensor(data[num_samples_start:num_samples_end,:], dtype=dtype).to(device)
data = torch.empty((num_samples,6+num_features), dtype=dtype).to(device)
data[:,6:] = data_org


# Magnitudes vel, acc, mass
temp, tmax = normalizer(data_org[:,4:7])
vel = denormalizer(torch.norm(temp, p=2, dim=1),tmax)
acc = torch.empty((num_samples,4), dtype=dtype).to(device)

temp,tmax = normalizer(data_org[:,7:10])
acc[:,0] = denormalizer(torch.norm(temp, p=2, dim=1),tmax)
temp,tmax = normalizer(data_org[:,10:13])
acc[:,1] = denormalizer(torch.norm(temp, p=2, dim=1),tmax)
temp,tmax = normalizer(data_org[:,13:16])
acc[:,2] = denormalizer(torch.norm(temp, p=2, dim=1),tmax)
temp,tmax = normalizer(data_org[:,16:19])
acc[:,3] = denormalizer(torch.norm(temp, p=2, dim=1),tmax)
mass = data_org[:,0]

data[:,0] = vel
data[:,1:5] = acc
data[:,5] = mass

# Normalize the data
eps = torch.finfo(torch.double).tiny
data_min = data[:,:6].min(axis=0, keepdim=True).values
data_max = data[:,:6].max(axis=0, keepdim=True).values
data[:,:6] = (data[:,:6] - data_min) / (data_max - data_min) +eps


# Wrap the feature tensor in a TensorDataset
# Each item from the dataset will be a tuple containing one tensor (X[i],)
dataset = TensorDataset(data)

# Define the proportion for the test set (e.g., 20%)
test_ratio = 0.2
num_test = int(num_samples * test_ratio)
num_train = num_samples - num_test

# Use random_split to split the dataset into train and test subsets
# Define generator with a fixed seed
generator = torch.Generator().manual_seed(42)
train_dataset, test_dataset = random_split(dataset, [num_train, num_test], generator=generator)

# Create DataLoaders for the training and testing datasets
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
print(data.shape)

torch.Size([18708, 68])


In [3]:
data_org[:,7:19].max()

tensor(3.3230e+43, device='cuda:0')

In [4]:
data_org[:,7:19]

tensor([[-5.1760e+07,  6.9440e+07,  1.9930e+08,  ...,  7.5800e+20,
          9.4280e+21,  2.6730e+22],
        [ 7.3060e+07, -9.5400e+07,  2.7010e+06,  ...,  3.9260e+21,
         -7.9120e+21, -2.3390e+21],
        [-6.8400e+07,  8.8380e+07, -1.0550e+08,  ..., -5.4870e+21,
          5.4520e+21, -1.0640e+22],
        ...,
        [-8.2630e+07, -2.7490e+08,  3.7450e+07,  ..., -1.1680e+22,
          1.3660e+22,  6.4800e+22],
        [ 1.0930e+08,  4.2830e+08, -2.7110e+08,  ..., -2.3880e+24,
         -1.5960e+25,  2.5750e+25],
        [-3.7300e+08, -1.4680e+09,  9.4750e+08,  ...,  8.5130e+24,
          5.3420e+25, -8.9590e+25]], device='cuda:0')

In [5]:
data[:,:6]

tensor([[ 6.9347e-03,  3.0354e-07,  2.8002e-12,  1.7239e-16,  7.9151e-22,
          2.1084e-01],
        [ 1.9106e-03,  1.7634e-08,  1.1221e-12,  3.9385e-17,  2.2971e-22,
          5.9583e-01],
        [ 6.5133e-04,  1.1625e-07,  1.3016e-12,  8.3355e-17,  3.4716e-22,
          4.4962e-01],
        ...,
        [ 8.8039e-03,  5.1604e-07,  1.0358e-11,  4.4025e-16,  1.9285e-21,
         2.2251e-308],
        [ 1.1908e-02,  1.1904e-06,  1.6754e-10,  1.8762e-14,  8.8836e-19,
          1.0000e+00],
        [ 4.5579e-02,  4.9237e-06,  5.9044e-10,  6.5928e-14,  3.0595e-18,
          2.5644e-01]], device='cuda:0')

In [6]:
data[:,:6]

tensor([[ 6.9347e-03,  3.0354e-07,  2.8002e-12,  1.7239e-16,  7.9151e-22,
          2.1084e-01],
        [ 1.9106e-03,  1.7634e-08,  1.1221e-12,  3.9385e-17,  2.2971e-22,
          5.9583e-01],
        [ 6.5133e-04,  1.1625e-07,  1.3016e-12,  8.3355e-17,  3.4716e-22,
          4.4962e-01],
        ...,
        [ 8.8039e-03,  5.1604e-07,  1.0358e-11,  4.4025e-16,  1.9285e-21,
         2.2251e-308],
        [ 1.1908e-02,  1.1904e-06,  1.6754e-10,  1.8762e-14,  8.8836e-19,
          1.0000e+00],
        [ 4.5579e-02,  4.9237e-06,  5.9044e-10,  6.5928e-14,  3.0595e-18,
          2.5644e-01]], device='cuda:0')

### Loss function test


In [ ]:
import structures
importlib.reload(structures)
from structures import *
import losses
importlib.reload(losses)
from losses import *
TargetEnergyError = 1e-5
#batch_size = 32
criterion = CustomizableLoss3DM(nParticle=3, nAttribute=20, nBatch=batch_size,alpha=0.1, beta=0.1, gamma=10.0, TargetEnergyError=TargetEnergyError,
                            data_min=data_min, data_max=data_max,device=device)
X = data[:20,:]
output = data[:20,[25,25]] #torch.zeros((X.shape[0],2)).to(device)
loss, loss_terms = criterion(output, X)
#output[:,0], X[:,-2], X[:,-1]

1e-05
tensor([3.2997e-04, 2.5714e+04, 3.4986e-04, 3.4986e-04, 8.9298e+04, 3.8232e-03,
        3.8232e-03, 3.6064e+05, 1.2763e-02, 1.2763e-02, 2.1405e+06, 6.9280e-03,
        6.9280e-03, 7.2961e+06, 2.3218e-02, 2.3218e-02, 3.8597e+07, 5.3466e-03,
        5.3466e-03, 1.1962e+08], device='cuda:0', grad_fn=<AbsBackward0>)


In [ ]:
data[:,25]#.mean()

tensor([6.2500e-06, 6.2500e-06, 6.2500e-06,  ..., 1.2500e-05, 2.4410e-08,
        2.4410e-08], device='cuda:0')

In [ ]:
data[train_dataset.indices,:3].min(dim=0), data[train_dataset.indices,:3].max(dim=0), data[train_dataset.indices,:3].mean(dim=0)

(torch.return_types.min(
 values=tensor([0., 0., 0.], device='cuda:0'),
 indices=tensor([5041,    9,    9], device='cuda:0')),
 torch.return_types.max(
 values=tensor([1., 1., 1.], device='cuda:0'),
 indices=tensor([1649, 1649, 1649], device='cuda:0')),
 tensor([0.2625, 0.1272, 0.0918], device='cuda:0'))

In [ ]:
data[test_dataset.indices,:3].min(dim=0), data[test_dataset.indices,:3].max(dim=0), data[test_dataset.indices,:3].mean(dim=0)

(torch.return_types.min(
 values=tensor([0.0003, 0.0000, 0.0000], device='cuda:0'),
 indices=tensor([368,  18,  18], device='cuda:0')),
 torch.return_types.max(
 values=tensor([0.9988, 0.9888, 0.9852], device='cuda:0'),
 indices=tensor([2865, 2865, 2865], device='cuda:0')),
 tensor([0.2565, 0.1216, 0.0869], device='cuda:0'))

In [ ]:
train_loader.

### Training

In [ ]:
import structures
importlib.reload(structures)
from structures import *
import losses
importlib.reload(losses)
from losses import *
import trainer
importlib.reload(trainer)
from trainer import *

input_mask = np.r_[0:6]
input_size = len(input_mask)
#hidden_dims = [32,64,64,32]     # Number of hidden neurons
hidden_dims = [8,16,16,4]     # Number of hidden neurons
#hidden_dims = [16,128,32,8]     # Number of hidden neurons
#hidden_dims = [8,8]     # Number of hidden neurons
output_size = 2     # Number of output 
TargetEnergyError = 1e-2
weights = {"time_step":0.1, "energy_loss":1}

# Instantiate the model
try: 
    del mode
    l
    print("model deleted")
except:
    pass


ds_config = {
    "train_batch_size": batch_size,
    "gradient_accumulation_steps": 1,
    "optimizer": {
        "type": "Adam",
        "params": {"lr": 1e-4, "betas": [0.9, 0.999], "eps": 1e-8}
    },
    "fp32": {
        "enabled": True  # Enables mixed precision training
    },
    "zero_optimization": {
        "stage": 1  # Enable ZeRO Stage 1 for memory optimization
    }
}


#model = SimpleNN(input_size, hidden_size, output_size).to(device)
model = FullyConnectedNN(input_dim=input_size, output_dim=output_size, hidden_dims=hidden_dims, activation='relu', dropout=0.0, output_positive=True).to(device)
#model = FullyConnectedNN(input_dim=input_size, output_dim=output_size, hidden_dims=hidden_dims, activation='relu', dropout=0.0, output_positive=False).to(device)


model_engine, optimizer, _, _ = deepspeed.initialize(
    model=model,
    model_parameters=model.parameters(),
    config=ds_config
)

# Define the loss function (CrossEntropyLoss is common for classification tasks)
criterion = CustomizableLoss3DMAdjusted(nParticle=3, nAttribute=20, nBatch=batch_size,alpha=weights['time_step'], beta=weights['energy_loss'], gamma=weights['energy_loss'], TargetEnergyError=TargetEnergyError,
                            data_min=data_min, data_max=data_max,device=device)

# Define the optimizer (Stochastic Gradient Descent in this example)
#optimizer = optim.SGD(model.parameters(), lr=0.001)
#optimizer = optim.Adam(model.parameters(), lr=1e-5)
#optimizer = optim.AdamW(model.parameters(), lr=0.00001)
#optimizer = optim.AdamW(model.parameters(), lr=0.00001)



# Paths to save best and last models
base_path = "/mnt/home/yjo10/projects/AITimeStepper/data/models/three_body/"
best_val_path = base_path + "best_model_loss_adjusted.pth"
best_timestep_path = base_path + "best_model_timestep_adjusted.pth"
#last_model_path = base_path + f"last_model_{epoch}.pth"
best_val_loss = float("inf")
best_largest_timestep = 0
with open(base_path+"c++/normalization_factors_adjusted.txt", "w") as f:
    f.write(f"{data_min[0][0]} {data_min[0][1]} {data_min[0][2]} {data_min[0][3]} {data_min[0][4]} {data_min[0][5]}\n")    
    f.write(f"{data_max[0][0]} {data_max[0][1]} {data_max[0][2]} {data_max[0][3]} {data_max[0][4]} {data_max[0][5]}\n")    

example_input = torch.randn(1, input_size).to(device)
exp_time_step = 1
exp_energy_loss = 1
# Training routine
num_epochs = 1000
lists = {"training_loss":[], "val_loss":[], "training_energy":[], "val_energy":[], "training_time_step":[], "val_time_step":[]}

for epoch in range(num_epochs):

    #train_loss, train_energy = \
    #    train_one_epoch(model, optimizer, criterion, train_loader, input_mask, weights, device)
    train_res = train_one_epoch_deepspeed(model_engine, optimizer, criterion, train_loader, input_mask, weights, device)
        
    #test_loss, energy_error, energy_error_std, energy_error_fiducial, energy_error_fiducial_std, energy_pred, energy_init, time_step, time_step_fiducial = \
    #    validate(model, criterion, test_loader, input_mask, weights, device)
    val_res = validate_deepspeed(model_engine, criterion, test_loader, input_mask, weights, device)

    lists["training_loss"].append(train_res["train_loss"])
    lists["training_energy"].append(train_res["energy_error"])
    lists["training_time_step"].append(train_res["time_step"])

    lists["val_loss"].append(val_res["val_loss"])
    lists["val_energy"].append(val_res["energy_error"])
    lists["val_time_step"].append(val_res["time_step"])

    if epoch > 5:
        # Save the best model based on validation loss
        if val_res['val_loss'] < best_val_loss:
            best_val_loss = val_res['val_loss']
            torch.save(model.state_dict(), best_val_path)
            #traced_model = torch.jit.trace(model, example_input)
            traced_model = torch.jit.script(model)
            traced_model.save(base_path + f"c++/best_model_loss_adjusted.pt")
            print(f"--> Best model saved at epoch {epoch} with val loss {best_val_loss:.4e}")
        
        # Save the best model based on validation largest timestep
        if val_res['time_step'] > best_largest_timestep:
            best_largest_timestep = val_res['time_step']
            torch.save(model.state_dict(), best_timestep_path)
            #traced_model = torch.jit.trace(model, example_input)
            traced_model = torch.jit.script(model)
            traced_model.save(base_path + f"c++/best_model_timestep_adjusted.pt")
            print(f"--> Best model saved at epoch {epoch} with largest timestep {best_largest_timestep:.4e}")

    # Save the last model after every epoch (or just once at the end)
    torch.save(model.state_dict(), base_path + f"model_{epoch}_adjusted.pth")
    #traced_model = torch.jit.trace(model, example_input)
    traced_model = torch.jit.script(model)
    traced_model.save(base_path + f"c++/model_{epoch}_adjusted.pt")



    fig, axes = pu.generateAxesForMultiplePlots(shape=(1,3),figsize=(10,10),hspace=0.1,wspace=0.1,
                                    gridspec=None)
    for i, (key, value) in enumerate(lists.items()):
        y = i//2
        axes[y].plot(range(epoch+1), value, label=key)
        axes[y].scatter(range(epoch+1), value)
        axes[y].set_yscale("log")


    axes[2].axhline(y=val_res['time_step_fiducial'], c='r',alpha=0.5,zorder=-1,label="Timestep Fiducial")
    axes[1].axhline(y=TargetEnergyError, c='r',alpha=0.5,zorder=-1,label="Target energy")
    #plt.legend(bbox_to_anchor=(1.01, 1), loc='upper left', borderaxespad=0.)
    axes[0].legend()
    axes[1].legend()
    axes[2].legend()
    #plt.tight_layout()
    plt.savefig("learning_curve_adjusted.png",dpi=100)
    plt.close()



    fig, axes = pu.generateAxesForMultiplePlots(shape=(1,2),figsize=(10,15),hspace=0.1,wspace=0.1,
                                    gridspec=None)
    # Get one batch from the validation loader
    dataiter = iter(train_loader)
    inputs, = next(dataiter)

    # Determine how many samples to take (if batch size < 50, take the whole batch)
    num_samples = min(50, batch_size)

    # Randomly select indices from the batch
    indices = random.sample(range(batch_size), num_samples)
    sampled_inputs = inputs[:,input_mask][indices]
    sampled_targets = inputs[indices,25]

    # Compute model predictions with no gradient computation
    with torch.no_grad():
        sampled_predictions = model(sampled_inputs)

    # Convert tensors to numpy arrays for plotting
    sampled_predictions = sampled_predictions[:,0].cpu().numpy().flatten()
    sampled_targets = sampled_targets.cpu().numpy().flatten()

    acc = inputs[:,1:5][indices]*(data_max[:,1:5]-data_min[:,1:5]) + data_min[:,1:5]
    dt_fid = torch.sqrt((acc[:,0]*acc[:,2]+acc[:,1]**2)/(acc[:,1]*acc[:,3]+acc[:,2]**2)).cpu().numpy().flatten()
    dt = sampled_predictions*dt_fid

    # Create sample indices for the x-axis (1st sample, 2nd sample, etc.)
    sample_indices = list(range(1, num_samples + 1))

    # Plot predictions and ground truth vs sample number
    # plt.figure(figsize=(8, 6))
    # Add vertical lines for each data point
    for x in sample_indices:
        axes[0].axvline(x=x, color='grey', linestyle='-', alpha=0.2,zorder=0)
    axes[0].scatter(sample_indices, sampled_targets, marker='o', label='Ground Truth', fc="none", ec="k",zorder=10)
    axes[0].scatter(sample_indices, dt, marker='s', label='Predictions', fc="none",ec="r" ,zorder=10)
    axes[0].scatter(sample_indices, dt_fid, marker='s', label='Fiducial', fc="none",ec="b" ,zorder=10)
    axes[0].set_yscale("log")
    axes[0].set_xlabel('Sample Number')
    axes[0].set_ylabel('Value')

    # Get one batch from the validation loader
    dataiter = iter(test_loader)
    inputs, = next(dataiter)

    # Determine how many samples to take (if batch size < 50, take the whole batch)
    num_samples = min(50, batch_size)

    # Randomly select indices from the batch
    indices = random.sample(range(batch_size), num_samples)
    sampled_inputs = inputs[:,input_mask][indices]
    sampled_targets = inputs[indices,25]

    # Compute model predictions with no gradient computation
    with torch.no_grad():
        sampled_predictions = model(sampled_inputs)

    # Convert tensors to numpy arrays for plotting
    sampled_predictions = sampled_predictions[:,0].cpu().numpy().flatten()
    sampled_targets = sampled_targets.cpu().numpy().flatten()

    acc = inputs[:,1:5][indices]*(data_max[:,1:5]-data_min[:,1:5]) + data_min[:,1:5]
    dt = torch.sqrt((acc[:,0]*acc[:,2]+acc[:,1]**2)/(acc[:,1]*acc[:,3]+acc[:,2]**2))
    dt = sampled_predictions*dt.cpu().numpy().flatten()

    # Create sample indices for the x-axis (1st sample, 2nd sample, etc.)
    sample_indices = list(range(1, num_samples + 1))

    # Plot predictions and ground truth vs sample number
    # plt.figure(figsize=(8, 6))
    # Add vertical lines for each data point
    for x in sample_indices:
        axes[1].axvline(x=x, color='grey', linestyle='-', alpha=0.2,zorder=0)
    axes[1].scatter(sample_indices, sampled_targets, marker='o', label='Ground Truth', fc="none", ec="k",zorder=10)
    axes[1].scatter(sample_indices, dt, marker='s', label='Predictions', fc="none",ec="r" ,zorder=10)
    axes[1].set_yscale("log")
    axes[1].set_xlabel('Sample Number')
    axes[1].set_ylabel('Value')

    axes[0].set_title(f'Epoch [{epoch}/{num_epochs}] Predictions and Ground Truth vs. Sample Number')
    #plt.legend()
    plt.legend(bbox_to_anchor=(1.01, 1), loc='upper left', borderaxespad=0.)
    plt.savefig("samples_adjusted.png")
    plt.close()


    """
    if time_step < time_step_fiducial:
        weights['time_step'] += 0.1*exp_time_step
        exp_time_step += 1
    else:
        weights['time_step'] += 0
        exp_time_step = 1

    if energy_error > energy_error_fiducial:
        weights['energy_loss'] += 0.1*exp_energy_loss
        exp_energy_loss += 1
    else:
        weights['energy_loss'] += 0
        exp_energy_loss = 1
        """

    #print(f"Epoch [{epoch}/{num_epochs}], Train Loss: {train_loss:.4e}, Test Loss: {test_loss:.4e}, Energy Loss: {energy_error:.4e}/{energy_error_fiducial:.4e}, std: {energy_error_std:.4e}/{energy_error_fiducial_std:.4e}, Time step: {time_step:.4e}/{time_step_fiducial:.4e}")
    #print(f"Epoch [{epoch}/{num_epochs}], Train Loss: {train_res['train_loss']:.4e}, Val Loss: {val_res['val_loss']:.4e}, Energy Loss: {np.log10(train_res['energy_error']):.4e}/{np.log10(train_res['energy_error_fiducial']):.4e}, {np.log10(val_res['energy_error']):.4e}/{np.log10(val_res['energy_error_fiducial']):.4e}, Time step: {train_res['time_step']:.4e}/{train_res['time_step_fiducial']:.4e}, {val_res['time_step']:.4e}/{val_res['time_step_fiducial']:.4e}, {val_res['time_step_std']:.4e}/{val_res['time_step_fiducial_std']:.4e}")
    print(f"Epoch [{epoch}/{num_epochs}], Timestep error: {train_res['time_step_relative_error']:.4e}/{val_res['time_step_relative_error']:.4e}, Energy Loss: {np.log10(train_res['energy_error']):.4e}/{np.log10(train_res['energy_error_fiducial']):.4e}, {np.log10(val_res['energy_error']):.4e}/{np.log10(val_res['energy_error_fiducial']):.4e}, Time step: {train_res['time_step']:.4e}/{train_res['time_step_fiducial']:.4e}, {val_res['time_step']:.4e}/{val_res['time_step_fiducial']:.4e}, {val_res['time_step_std']:.4e}/{val_res['time_step_fiducial_std']:.4e}")
    #print(f"Epoch [{epoch}/{num_epochs}], Train Loss: {train_res['train_loss']:.4e}, Val Loss: {val_res['val_loss']:.4e}, Energy Loss: {np.log10(train_res['energy_error']):.4e}/{np.log10(train_res['energy_error_fiducial']):.4e}, {np.log10(val_res['energy_error']):.4e}/{np.log10(val_res['energy_error_fiducial']):.4e}, energy_pred: {energy_pred:.4e}/{energy_init:.4e}, Time step: {time_step:.4e}/{time_step_fiducial:.4e}")

[2025-04-07 14:55:38,715] [INFO] [logging.py:128:log_dist] [Rank 0] DeepSpeed info: version=0.16.4, git-hash=unknown, git-branch=unknown
[2025-04-07 14:55:38,715] [INFO] [config.py:734:__init__] Config mesh_device None world_size = 1
[2025-04-07 14:55:38,718] [INFO] [logging.py:128:log_dist] [Rank 0] DeepSpeed Flops Profiler Enabled: False
[2025-04-07 14:55:38,719] [INFO] [logging.py:128:log_dist] [Rank 0] Using DeepSpeed Optimizer param name adam as basic optimizer
[2025-04-07 14:55:38,719] [INFO] [logging.py:128:log_dist] [Rank 0] Removing param_group that has no 'params' in the basic Optimizer
[2025-04-07 14:55:38,720] [INFO] [logging.py:128:log_dist] [Rank 0] DeepSpeed Basic Optimizer = FusedAdam
[2025-04-07 14:55:38,720] [INFO] [utils.py:59:is_zero_supported_optimizer] Checking ZeRO support for optimizer=FusedAdam type=<class 'deepspeed.ops.adam.fused_adam.FusedAdam'>
[2025-04-07 14:55:38,720] [INFO] [logging.py:128:log_dist] [Rank 0] Creating torch.float32 ZeRO stage 1 optimizer


[2025-04-07 14:55:39,729] [INFO] [utils.py:781:see_memory_usage] Before initializing optimizer states
[2025-04-07 14:55:39,732] [INFO] [utils.py:782:see_memory_usage] MA 3.76 GB         Max_MA 7.49 GB         CA 7.5 GB         Max_CA 7 GB 
[2025-04-07 14:55:39,734] [INFO] [utils.py:789:see_memory_usage] CPU Virtual Memory:  used = 69.74 GB, percent = 6.9%
[2025-04-07 14:55:40,009] [INFO] [utils.py:781:see_memory_usage] After initializing optimizer states
[2025-04-07 14:55:40,011] [INFO] [utils.py:782:see_memory_usage] MA 3.76 GB         Max_MA 3.76 GB         CA 7.5 GB         Max_CA 7 GB 
[2025-04-07 14:55:40,011] [INFO] [utils.py:789:see_memory_usage] CPU Virtual Memory:  used = 69.74 GB, percent = 6.9%
[2025-04-07 14:55:40,012] [INFO] [stage_1_and_2.py:550:__init__] optimizer state initialized
[2025-04-07 14:55:40,280] [INFO] [utils.py:781:see_memory_usage] After initializing ZeRO optimizer
[2025-04-07 14:55:40,281] [INFO] [utils.py:782:see_memory_usage] MA 3.76 GB         Max_MA 3.

# Two Body With Tidal Fields

In [17]:
torch.__version__

'2.2.2'